# Debugging autoreload

In [ ]:
%load_ext autoreload
%autoreload 2

# Load packages

In [1]:
import pathlib
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import statsmodels.formula.api as smf

from functools import reduce
from plottable import Table, ColumnDefinition
from statsmodels.stats.multitest import multipletests
from scipy.stats import pearsonr, mannwhitneyu, levene, variation, spearmanr

# Calculate scores

## Load data

In [2]:
path = "E:/YandexDisk/bbd/fmba"
path_save = "E:/YandexDisk/bbd/fmba/05_scores"

df = pd.read_excel(f"{path}/pheno_fixed.xlsx", index_col=0)
df.index = df.index.astype(str)

df['дата рождения'] = pd.to_datetime(df['дата рождения'])
df['Age'] = (pd.to_datetime("2024-11-11") - df['дата рождения']) / np.timedelta64(1, 'D') / 365.25

df['smoking'] = df['терапевт - фактор_риска'].str.contains('Курение табака')
df['smoking'] = df['smoking'].replace({True: 1, False: 0})

df['hypertension'] = np.where((df["Терапевт, I10"] == 1) | (df["Терапевт, I10.0"] == 1) | (df["Терапевт, I11"] == 1) | (df["Терапевт, I11.0"] == 1) | (df["Терапевт, I11.9"] == 1), 1, 0)

C:\Users\alena\AppData\Local\Temp\ipykernel_38988\3723450303.py:11: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df['smoking'] = df['smoking'].replace({True: 1, False: 0})


## SCORE

In [ ]:
cols_for_score = ["терапевт - артериальное давление верхнее",
                  "биохимический анализ крови - холестерин",
                  "smoking",
                  "Age"
]
for col in cols_for_score:
    df[col] = df[col].replace('нет', np.nan)
    df[col] = df[col].replace('норма', np.nan)
    df[col] = pd.to_numeric(df[col])

chd_s0_age = np.exp(-np.exp(-21.0) * np.power((df['Age'] - 20.0), 4.62))
chd_s0_age_10 = np.exp(-np.exp(-21.0) * np.power((df['Age'] - 10.0), 4.62))
chd_w = 0.24 * (df['биохимический анализ крови - холестерин'] - 6.0) + 0.018 * (df['терапевт - артериальное давление верхнее'] - 120.0) + 0.71 * df['smoking']
chd_s_age = np.power(chd_s0_age, np.exp(chd_w))
chd_s_age_10 = np.power(chd_s0_age_10, np.exp(chd_w))
chd_s10_age = chd_s_age_10 / chd_s_age
chd_risk_10 = 1.0 - chd_s10_age

nchd_s0_age = np.exp(-np.exp(-25.7) * np.power((df['Age'] - 20.0), 5.47))
nchd_s0_age_10 = np.exp(-np.exp(-25.7) * np.power((df['Age'] - 10.0), 5.47))
nchd_w = 0.02 * (df['биохимический анализ крови - холестерин'] - 6.0) + 0.022 * (df['терапевт - артериальное давление верхнее'] - 120.0) + 0.63 * df['smoking']
nchd_s_age = np.power(nchd_s0_age, np.exp(nchd_w))
nchd_s_age_10 = np.power(nchd_s0_age_10, np.exp(nchd_w))
nchd_s10_age = nchd_s_age_10 / nchd_s_age
nchd_risk_10 = 1.0 - nchd_s10_age

risk_10_age = chd_risk_10 + nchd_risk_10

df['SCORE'] = risk_10_age
df_res_score = df[['SCORE']].copy()

In [ ]:
threshold = 0.01
df_res_score[f'SCORE group thr={threshold}'] = df_res_score['SCORE']
df_res_score[f'SCORE group thr={threshold}'].mask(df_res_score['SCORE'] >= threshold, 'High', inplace=True)
df_res_score[f'SCORE group thr={threshold}'].mask(df_res_score['SCORE'] < threshold, 'Low', inplace=True)
threshold = 0.02
df_res_score[f'SCORE group thr={threshold}'] = df_res_score['SCORE']
df_res_score[f'SCORE group thr={threshold}'].mask(df_res_score['SCORE'] >= threshold, 'High', inplace=True)
df_res_score[f'SCORE group thr={threshold}'].mask(df_res_score['SCORE'] < threshold, 'Low', inplace=True)

In [ ]:
df_res_score.to_excel(f'{path_save}/df_scores.xlsx')

## SCORE2

### Without HDL cholesterol

In [ ]:
cols_for_score = ["терапевт - артериальное давление верхнее",
                  "биохимический анализ крови - холестерин",
                  "smoking",
                  "Age"
]
for col in cols_for_score:
    df[col] = df[col].replace('нет', np.nan)
    df[col] = df[col].replace('норма', np.nan)
    df[col] = pd.to_numeric(df[col])

lincomb = 0.3742 * (df['Age'] - 60.0) / 5.0 + \
      0.1458 * (df['биохимический анализ крови - холестерин'] - 6.0) - \
      0.2698 * 0.0 + \
      0.2777 * (df['терапевт - артериальное давление верхнее'] - 120.0) / 20.0 + \
      0.6457 * 0.0 + \
      0.6012 * df['smoking'] - \
      0.0281 * ((df['Age'] - 60.0) / 5.0) * (df['биохимический анализ крови - холестерин'] - 6.0) + \
      0.0426 * ((df['Age'] - 60.0) / 5.0) * 0.0 - \
      0.0255 * ((df['Age'] - 60.0) / 5.0) * ((df['терапевт - артериальное давление верхнее'] - 120.0) / 20.0) - \
      0.0983 * ((df['Age'] - 60.0) / 5.0) * 0.0 -\
      0.0755 * ((df['Age'] - 60.0) / 5.0) * df['smoking']

risk_10 = 1.0 - np.power(0.9605, np.exp(lincomb))
risk_10_calib = 1.0 - np.exp(-np.exp(0.5836 + 0.8294 * np.log(-np.log(1.0 - risk_10))))

df['SCORE2 wo HDL'] = risk_10_calib
df_res_score = df[['SCORE2 wo HDL']].copy()

In [ ]:
threshold = 0.05
df_res_score[f'SCORE2 wo HDL group thr={threshold}'] = df_res_score['SCORE2 wo HDL']
df_res_score[f'SCORE2 wo HDL group thr={threshold}'].mask(df_res_score['SCORE2 wo HDL'] >= threshold, 'High', inplace=True)
df_res_score[f'SCORE2 wo HDL group thr={threshold}'].mask(df_res_score['SCORE2 wo HDL'] < threshold, 'Low', inplace=True)

In [ ]:
df_scores = pd.read_excel(f"{path}/05_scores/df_scores.xlsx", index_col=0)
df_scores.index = df_scores.index.astype(str)
df_scores = pd.concat([df_scores, df_res_score], axis=1)
df_scores.to_excel(f'{path_save}/df_scores.xlsx')

### With population-average HDL cholesterol

In [ ]:
cols_for_score = ["терапевт - артериальное давление верхнее",
                  "биохимический анализ крови - холестерин",
                  "smoking",
                  "Age"
]
for col in cols_for_score:
    df[col] = df[col].replace('нет', np.nan)
    df[col] = df[col].replace('норма', np.nan)
    df[col] = pd.to_numeric(df[col])

lincomb = 0.3742 * (df['Age'] - 60.0) / 5.0 + \
      0.1458 * (df['биохимический анализ крови - холестерин'] - 6.0) - \
      0.2698 * (1.0 - 1.3) + \
      0.2777 * (df['терапевт - артериальное давление верхнее'] - 120.0) / 20.0 + \
      0.6457 * 0.0 + \
      0.6012 * df['smoking'] - \
      0.0281 * ((df['Age'] - 60.0) / 5.0) * (df['биохимический анализ крови - холестерин'] - 6.0) + \
      0.0426 * ((df['Age'] - 60.0) / 5.0) * (1.0 - 1.3) - \
      0.0255 * ((df['Age'] - 60.0) / 5.0) * ((df['терапевт - артериальное давление верхнее'] - 120.0) / 20.0) - \
      0.0983 * ((df['Age'] - 60.0) / 5.0) * 0.0 -\
      0.0755 * ((df['Age'] - 60.0) / 5.0) * df['smoking']

risk_10 = 1.0 - np.power(0.9605, np.exp(lincomb))
risk_10_calib = 1.0 - np.exp(-np.exp(0.5836 + 0.8294 * np.log(-np.log(1.0 - risk_10))))

df['SCORE2 aver HDL'] = risk_10_calib
df_res_score = df[['SCORE2 aver HDL']].copy()

In [ ]:
threshold = 0.05
df_res_score[f'SCORE2 aver HDL group thr={threshold}'] = df_res_score['SCORE2 aver HDL']
df_res_score[f'SCORE2 aver HDL group thr={threshold}'].mask(df_res_score['SCORE2 aver HDL'] >= threshold, 'High', inplace=True)
df_res_score[f'SCORE2 aver HDL group thr={threshold}'].mask(df_res_score['SCORE2 aver HDL'] < threshold, 'Low', inplace=True)

In [ ]:
df_scores = pd.read_excel(f"{path}/05_scores/df_scores.xlsx", index_col=0)
df_scores.index = df_scores.index.astype(str)
df_scores = pd.concat([df_scores, df_res_score], axis=1)
df_scores.to_excel(f'{path_save}/df_scores.xlsx')

## Framingham Risk Score

### Without HDL cholesterol

In [ ]:
cols_for_score = ["терапевт - артериальное давление верхнее",
                  "биохимический анализ крови - холестерин",
                  "smoking",
                  "hypertension",
                  "Age"
]
for col in cols_for_score:
    df[col] = df[col].replace('нет', np.nan)
    df[col] = df[col].replace('норма', np.nan)
    df[col] = pd.to_numeric(df[col])

df['FHR wo HDL, points'] = 0.0

for index in df.index:
    # Age
    if df.loc[index,'Age'] < 35.0: 
        df.loc[index,'FHR wo HDL, points'] += 0.0
    if df.loc[index,'Age'] >= 35.0 and df.loc[index,'Age'] < 40.0: 
        df.loc[index,'FHR wo HDL, points'] += 2.0
    if df.loc[index,'Age'] >= 40.0 and df.loc[index,'Age'] < 45.0: 
        df.loc[index,'FHR wo HDL, points'] += 5.0
    if df.loc[index,'Age'] >= 45.0 and df.loc[index,'Age'] < 50.0: 
        df.loc[index,'FHR wo HDL, points'] += 6.0
    if df.loc[index,'Age'] >= 50.0 and df.loc[index,'Age'] < 55.0: 
        df.loc[index,'FHR wo HDL, points'] += 8.0 
    if df.loc[index,'Age'] >= 55.0 and df.loc[index,'Age'] < 60.0: 
        df.loc[index,'FHR wo HDL, points'] += 10.0 
    if df.loc[index,'Age'] >= 60.0 and df.loc[index,'Age'] < 65.0: 
        df.loc[index,'FHR wo HDL, points'] += 11.0 
    if df.loc[index,'Age'] >= 65.0 and df.loc[index,'Age'] < 70.0: 
        df.loc[index,'FHR wo HDL, points'] += 12.0 
    if df.loc[index,'Age'] >= 70.0 and df.loc[index,'Age'] < 75.0: 
        df.loc[index,'FHR wo HDL, points'] += 14.0 
    if df.loc[index,'Age'] >= 75.0: 
        df.loc[index,'FHR wo HDL, points'] += 15.0
    # Total cholesterol
    if df.loc[index,"биохимический анализ крови - холестерин"] < 4.1: 
        df.loc[index,'FHR wo HDL, points'] += 0.0
    if df.loc[index,"биохимический анализ крови - холестерин"] >= 4.1 and df.loc[index,"биохимический анализ крови - холестерин"] < 5.2: 
        df.loc[index,'FHR wo HDL, points'] += 1.0
    if df.loc[index,"биохимический анализ крови - холестерин"] >= 5.2 and df.loc[index,"биохимический анализ крови - холестерин"] < 6.2: 
        df.loc[index,'FHR wo HDL, points'] += 2.0
    if df.loc[index,"биохимический анализ крови - холестерин"] >= 6.2 and df.loc[index,"биохимический анализ крови - холестерин"] < 7.2: 
        df.loc[index,'FHR wo HDL, points'] += 3.0
    if df.loc[index,"биохимический анализ крови - холестерин"] >= 7.2: 
        df.loc[index,'FHR wo HDL, points'] += 4.0
    # Blood pressure
    if df.loc[index,"терапевт - артериальное давление верхнее"] < 120.0 and df.loc[index,"hypertension"] == 0: 
        df.loc[index,'FHR wo HDL, points'] -= 2.0
    if df.loc[index,"терапевт - артериальное давление верхнее"] < 120.0 and df.loc[index,"hypertension"] == 1: 
        df.loc[index,'FHR wo HDL, points'] += 0.0
    if df.loc[index,"терапевт - артериальное давление верхнее"] >= 120.0 and df.loc[index,"терапевт - артериальное давление верхнее"] < 130.0 and df.loc[index,"hypertension"] == 0: 
        df.loc[index,'FHR wo HDL, points'] += 0.0
    if df.loc[index,"терапевт - артериальное давление верхнее"] >= 120.0 and df.loc[index,"терапевт - артериальное давление верхнее"] < 130.0 and df.loc[index,"hypertension"] == 1: 
        df.loc[index,'FHR wo HDL, points'] += 2.0
    if df.loc[index,"терапевт - артериальное давление верхнее"] >= 130.0 and df.loc[index,"терапевт - артериальное давление верхнее"] < 140.0 and df.loc[index,"hypertension"] == 0: 
        df.loc[index,'FHR wo HDL, points'] += 1.0
    if df.loc[index,"терапевт - артериальное давление верхнее"] >= 130.0 and df.loc[index,"терапевт - артериальное давление верхнее"] < 140.0 and df.loc[index,"hypertension"] == 1: 
        df.loc[index,'FHR wo HDL, points'] += 3.0
    if df.loc[index,"терапевт - артериальное давление верхнее"] >= 140.0 and df.loc[index,"терапевт - артериальное давление верхнее"] < 150.0 and df.loc[index,"hypertension"] == 0: 
        df.loc[index,'FHR wo HDL, points'] += 2.0
    if df.loc[index,"терапевт - артериальное давление верхнее"] >= 140.0 and df.loc[index,"терапевт - артериальное давление верхнее"] < 150.0 and df.loc[index,"hypertension"] == 1: 
        df.loc[index,'FHR wo HDL, points'] += 4.0
    if df.loc[index,"терапевт - артериальное давление верхнее"] >= 150.0 and df.loc[index,"терапевт - артериальное давление верхнее"] < 160.0 and df.loc[index,"hypertension"] == 0: 
        df.loc[index,'FHR wo HDL, points'] += 2.0
    if df.loc[index,"терапевт - артериальное давление верхнее"] >= 150.0 and df.loc[index,"терапевт - артериальное давление верхнее"] < 160.0 and df.loc[index,"hypertension"] == 1: 
        df.loc[index,'FHR wo HDL, points'] += 4.0
    if df.loc[index,"терапевт - артериальное давление верхнее"] >= 160.0 and df.loc[index,"hypertension"] == 0: 
        df.loc[index,'FHR wo HDL, points'] += 3.0
    if df.loc[index,"терапевт - артериальное давление верхнее"] >= 160.0 and df.loc[index,"hypertension"] == 1: 
        df.loc[index,'FHR wo HDL, points'] += 5.0
    # Smoking
    if df.loc[index,"smoking"] == 0: 
        df.loc[index,'FHR wo HDL, points'] += 0.0
    if df.loc[index,"smoking"] == 1: 
        df.loc[index,'FHR wo HDL, points'] += 4.0
    # Recalculate to risk
    if df.loc[index,'FHR wo HDL, points'] <= -1.0:
        df.loc[index,'FHR wo HDL'] = 1.0
    if df.loc[index,'FHR wo HDL, points'] == 0.0:
        df.loc[index,'FHR wo HDL'] = 1.1
    if df.loc[index,'FHR wo HDL, points'] == 1.0:
        df.loc[index,'FHR wo HDL'] = 1.4
    if df.loc[index,'FHR wo HDL, points'] == 2.0:
        df.loc[index,'FHR wo HDL'] = 1.6
    if df.loc[index,'FHR wo HDL, points'] == 3.0:
        df.loc[index,'FHR wo HDL'] = 1.9
    if df.loc[index,'FHR wo HDL, points'] == 4.0:
        df.loc[index,'FHR wo HDL'] = 2.3
    if df.loc[index,'FHR wo HDL, points'] == 5.0:
        df.loc[index,'FHR wo HDL'] = 2.8
    if df.loc[index,'FHR wo HDL, points'] == 6.0:
        df.loc[index,'FHR wo HDL'] = 3.3
    if df.loc[index,'FHR wo HDL, points'] == 7.0:
        df.loc[index,'FHR wo HDL'] = 3.9
    if df.loc[index,'FHR wo HDL, points'] == 8.0:
        df.loc[index,'FHR wo HDL'] = 4.7
    if df.loc[index,'FHR wo HDL, points'] == 9.0:
        df.loc[index,'FHR wo HDL'] = 5.6
    if df.loc[index,'FHR wo HDL, points'] == 10.0:
        df.loc[index,'FHR wo HDL'] = 6.7
    if df.loc[index,'FHR wo HDL, points'] == 11.0:
        df.loc[index,'FHR wo HDL'] = 7.9
    if df.loc[index,'FHR wo HDL, points'] == 12.0:
        df.loc[index,'FHR wo HDL'] = 9.4
    if df.loc[index,'FHR wo HDL, points'] == 13.0:
        df.loc[index,'FHR wo HDL'] = 11.2
    if df.loc[index,'FHR wo HDL, points'] == 14.0:
        df.loc[index,'FHR wo HDL'] = 13.2
    if df.loc[index,'FHR wo HDL, points'] == 15.0:
        df.loc[index,'FHR wo HDL'] = 15.6
    if df.loc[index,'FHR wo HDL, points'] == 16.0:
        df.loc[index,'FHR wo HDL'] = 18.4
    if df.loc[index,'FHR wo HDL, points'] == 17.0:
        df.loc[index,'FHR wo HDL'] = 21.6
    if df.loc[index,'FHR wo HDL, points'] == 18.0:
        df.loc[index,'FHR wo HDL'] = 25.3
    if df.loc[index,'FHR wo HDL, points'] == 19.0:
        df.loc[index,'FHR wo HDL'] = 29.4
    if df.loc[index,'FHR wo HDL, points'] > 19.0:
        df.loc[index,'FHR wo HDL'] = 30.0
    # Recalculate to heart age
    if df.loc[index,'FHR wo HDL, points'] <= 0.0:
        df.loc[index,'FHR wo HDL Heart Age'] = 30.0
    if df.loc[index,'FHR wo HDL, points'] == 1.0:
        df.loc[index,'FHR wo HDL Heart Age'] = 32.0
    if df.loc[index,'FHR wo HDL, points'] == 2.0:
        df.loc[index,'FHR wo HDL Heart Age'] = 34.0
    if df.loc[index,'FHR wo HDL, points'] == 3.0:
        df.loc[index,'FHR wo HDL Heart Age'] = 36.0
    if df.loc[index,'FHR wo HDL, points'] == 4.0:
        df.loc[index,'FHR wo HDL Heart Age'] = 38.0
    if df.loc[index,'FHR wo HDL, points'] == 5.0:
        df.loc[index,'FHR wo HDL Heart Age'] = 40.0
    if df.loc[index,'FHR wo HDL, points'] == 6.0:
        df.loc[index,'FHR wo HDL Heart Age'] = 42.0
    if df.loc[index,'FHR wo HDL, points'] == 7.0:
        df.loc[index,'FHR wo HDL Heart Age'] = 45.0
    if df.loc[index,'FHR wo HDL, points'] == 8.0:
        df.loc[index,'FHR wo HDL Heart Age'] = 48.0
    if df.loc[index,'FHR wo HDL, points'] == 9.0:
        df.loc[index,'FHR wo HDL Heart Age'] = 51.0
    if df.loc[index,'FHR wo HDL, points'] == 10.0:
        df.loc[index,'FHR wo HDL Heart Age'] = 54.0
    if df.loc[index,'FHR wo HDL, points'] == 11.0:
        df.loc[index,'FHR wo HDL Heart Age'] = 57.0
    if df.loc[index,'FHR wo HDL, points'] == 12.0:
        df.loc[index,'FHR wo HDL Heart Age'] = 60.0
    if df.loc[index,'FHR wo HDL, points'] == 13.0:
        df.loc[index,'FHR wo HDL Heart Age'] = 64.0
    if df.loc[index,'FHR wo HDL, points'] == 14.0:
        df.loc[index,'FHR wo HDL Heart Age'] = 68.0
    if df.loc[index,'FHR wo HDL, points'] == 15.0:
        df.loc[index,'FHR wo HDL Heart Age'] = 72.0
    if df.loc[index,'FHR wo HDL, points'] == 16.0:
        df.loc[index,'FHR wo HDL Heart Age'] = 76.0
    if df.loc[index,'FHR wo HDL, points'] >= 17.0:
        df.loc[index,'FHR wo HDL Heart Age'] = 80.0

df_res_score = df[['FHR wo HDL', 'FHR wo HDL Heart Age']].copy()

In [ ]:
threshold = 10.0
df_res_score[f'FHR wo HDL group thr={threshold}'] = df_res_score['FHR wo HDL']
df_res_score[f'FHR wo HDL group thr={threshold}'].mask(df_res_score['FHR wo HDL'] >= threshold, 'High', inplace=True)
df_res_score[f'FHR wo HDL group thr={threshold}'].mask(df_res_score['FHR wo HDL'] < threshold, 'Low', inplace=True)

In [ ]:
df_res_score['FHR wo HDL Heart Age Acceleration'] = df_res_score['FHR wo HDL Heart Age'] - df['Age']
threshold = 0.0
df_res_score[f'FHR wo HDL Heart Age Acceleration group thr={threshold}'] = df_res_score['FHR wo HDL Heart Age Acceleration']
df_res_score[f'FHR wo HDL Heart Age Acceleration group thr={threshold}'].mask(df_res_score['FHR wo HDL Heart Age Acceleration'] >= threshold, 'High', inplace=True)
df_res_score[f'FHR wo HDL Heart Age Acceleration group thr={threshold}'].mask(df_res_score['FHR wo HDL Heart Age Acceleration'] < threshold, 'Low', inplace=True)

In [ ]:
df_scores = pd.read_excel(f"{path}/05_scores/df_scores.xlsx", index_col=0)
df_scores.index = df_scores.index.astype(str)
df_scores = pd.concat([df_scores, df_res_score], axis=1)
df_scores.to_excel(f'{path_save}/df_scores.xlsx')

### With population-average HDL cholesterol

In [ ]:
cols_for_score = ["терапевт - артериальное давление верхнее",
                  "биохимический анализ крови - холестерин",
                  "smoking",
                  "hypertension",
                  "Age"
]
for col in cols_for_score:
    df[col] = df[col].replace('нет', np.nan)
    df[col] = df[col].replace('норма', np.nan)
    df[col] = pd.to_numeric(df[col])

df['FHR aver HDL, points'] = 0.0

for index in df.index:
    # Age
    if df.loc[index,'Age'] < 35.0: 
        df.loc[index,'FHR aver HDL, points'] += 0.0
    if df.loc[index,'Age'] >= 35.0 and df.loc[index,'Age'] < 40.0: 
        df.loc[index,'FHR aver HDL, points'] += 2.0
    if df.loc[index,'Age'] >= 40.0 and df.loc[index,'Age'] < 45.0: 
        df.loc[index,'FHR aver HDL, points'] += 5.0
    if df.loc[index,'Age'] >= 45.0 and df.loc[index,'Age'] < 50.0: 
        df.loc[index,'FHR aver HDL, points'] += 6.0
    if df.loc[index,'Age'] >= 50.0 and df.loc[index,'Age'] < 55.0: 
        df.loc[index,'FHR aver HDL, points'] += 8.0 
    if df.loc[index,'Age'] >= 55.0 and df.loc[index,'Age'] < 60.0: 
        df.loc[index,'FHR aver HDL, points'] += 10.0 
    if df.loc[index,'Age'] >= 60.0 and df.loc[index,'Age'] < 65.0: 
        df.loc[index,'FHR aver HDL, points'] += 11.0 
    if df.loc[index,'Age'] >= 65.0 and df.loc[index,'Age'] < 70.0: 
        df.loc[index,'FHR aver HDL, points'] += 12.0 
    if df.loc[index,'Age'] >= 70.0 and df.loc[index,'Age'] < 75.0: 
        df.loc[index,'FHR aver HDL, points'] += 14.0 
    if df.loc[index,'Age'] >= 75.0: 
        df.loc[index,'FHR aver HDL, points'] += 15.0
    # HDL cholesterol
    df.loc[index,'FHR aver HDL, points'] += 1.0
    # Total cholesterol
    if df.loc[index,"биохимический анализ крови - холестерин"] < 4.1: 
        df.loc[index,'FHR aver HDL, points'] += 0.0
    if df.loc[index,"биохимический анализ крови - холестерин"] >= 4.1 and df.loc[index,"биохимический анализ крови - холестерин"] < 5.2: 
        df.loc[index,'FHR aver HDL, points'] += 1.0
    if df.loc[index,"биохимический анализ крови - холестерин"] >= 5.2 and df.loc[index,"биохимический анализ крови - холестерин"] < 6.2: 
        df.loc[index,'FHR aver HDL, points'] += 2.0
    if df.loc[index,"биохимический анализ крови - холестерин"] >= 6.2 and df.loc[index,"биохимический анализ крови - холестерин"] < 7.2: 
        df.loc[index,'FHR aver HDL, points'] += 3.0
    if df.loc[index,"биохимический анализ крови - холестерин"] >= 7.2: 
        df.loc[index,'FHR aver HDL, points'] += 4.0
    # Blood pressure
    if df.loc[index,"терапевт - артериальное давление верхнее"] < 120.0 and df.loc[index,"hypertension"] == 0: 
        df.loc[index,'FHR aver HDL, points'] -= 2.0
    if df.loc[index,"терапевт - артериальное давление верхнее"] < 120.0 and df.loc[index,"hypertension"] == 1: 
        df.loc[index,'FHR aver HDL, points'] += 0.0
    if df.loc[index,"терапевт - артериальное давление верхнее"] >= 120.0 and df.loc[index,"терапевт - артериальное давление верхнее"] < 130.0 and df.loc[index,"hypertension"] == 0: 
        df.loc[index,'FHR aver HDL, points'] += 0.0
    if df.loc[index,"терапевт - артериальное давление верхнее"] >= 120.0 and df.loc[index,"терапевт - артериальное давление верхнее"] < 130.0 and df.loc[index,"hypertension"] == 1: 
        df.loc[index,'FHR aver HDL, points'] += 2.0
    if df.loc[index,"терапевт - артериальное давление верхнее"] >= 130.0 and df.loc[index,"терапевт - артериальное давление верхнее"] < 140.0 and df.loc[index,"hypertension"] == 0: 
        df.loc[index,'FHR aver HDL, points'] += 1.0
    if df.loc[index,"терапевт - артериальное давление верхнее"] >= 130.0 and df.loc[index,"терапевт - артериальное давление верхнее"] < 140.0 and df.loc[index,"hypertension"] == 1: 
        df.loc[index,'FHR aver HDL, points'] += 3.0
    if df.loc[index,"терапевт - артериальное давление верхнее"] >= 140.0 and df.loc[index,"терапевт - артериальное давление верхнее"] < 150.0 and df.loc[index,"hypertension"] == 0: 
        df.loc[index,'FHR aver HDL, points'] += 2.0
    if df.loc[index,"терапевт - артериальное давление верхнее"] >= 140.0 and df.loc[index,"терапевт - артериальное давление верхнее"] < 150.0 and df.loc[index,"hypertension"] == 1: 
        df.loc[index,'FHR aver HDL, points'] += 4.0
    if df.loc[index,"терапевт - артериальное давление верхнее"] >= 150.0 and df.loc[index,"терапевт - артериальное давление верхнее"] < 160.0 and df.loc[index,"hypertension"] == 0: 
        df.loc[index,'FHR aver HDL, points'] += 2.0
    if df.loc[index,"терапевт - артериальное давление верхнее"] >= 150.0 and df.loc[index,"терапевт - артериальное давление верхнее"] < 160.0 and df.loc[index,"hypertension"] == 1: 
        df.loc[index,'FHR aver HDL, points'] += 4.0
    if df.loc[index,"терапевт - артериальное давление верхнее"] >= 160.0 and df.loc[index,"hypertension"] == 0: 
        df.loc[index,'FHR aver HDL, points'] += 3.0
    if df.loc[index,"терапевт - артериальное давление верхнее"] >= 160.0 and df.loc[index,"hypertension"] == 1: 
        df.loc[index,'FHR aver HDL, points'] += 5.0
    # Smoking
    if df.loc[index,"smoking"] == 0: 
        df.loc[index,'FHR aver HDL, points'] += 0.0
    if df.loc[index,"smoking"] == 1: 
        df.loc[index,'FHR aver HDL, points'] += 4.0
    # Recalculate to risk
    if df.loc[index,'FHR aver HDL, points'] <= -3.0:
        df.loc[index,'FHR aver HDL'] = 1.0
    if df.loc[index,'FHR aver HDL, points'] == -2.0:
        df.loc[index,'FHR aver HDL'] = 1.1
    if df.loc[index,'FHR aver HDL, points'] == -1.0:
        df.loc[index,'FHR aver HDL'] = 1.4
    if df.loc[index,'FHR aver HDL, points'] == 0.0:
        df.loc[index,'FHR aver HDL'] = 1.6
    if df.loc[index,'FHR aver HDL, points'] == 1.0:
        df.loc[index,'FHR aver HDL'] = 1.9
    if df.loc[index,'FHR aver HDL, points'] == 2.0:
        df.loc[index,'FHR aver HDL'] = 2.3
    if df.loc[index,'FHR aver HDL, points'] == 3.0:
        df.loc[index,'FHR aver HDL'] = 2.8
    if df.loc[index,'FHR aver HDL, points'] == 4.0:
        df.loc[index,'FHR aver HDL'] = 3.3
    if df.loc[index,'FHR aver HDL, points'] == 5.0:
        df.loc[index,'FHR aver HDL'] = 3.9
    if df.loc[index,'FHR aver HDL, points'] == 6.0:
        df.loc[index,'FHR aver HDL'] = 4.7
    if df.loc[index,'FHR aver HDL, points'] == 7.0:
        df.loc[index,'FHR aver HDL'] = 5.6
    if df.loc[index,'FHR aver HDL, points'] == 8.0:
        df.loc[index,'FHR aver HDL'] = 6.7
    if df.loc[index,'FHR aver HDL, points'] == 9.0:
        df.loc[index,'FHR aver HDL'] = 7.9
    if df.loc[index,'FHR aver HDL, points'] == 10.0:
        df.loc[index,'FHR aver HDL'] = 9.4
    if df.loc[index,'FHR aver HDL, points'] == 11.0:
        df.loc[index,'FHR aver HDL'] = 11.2
    if df.loc[index,'FHR aver HDL, points'] == 12.0:
        df.loc[index,'FHR aver HDL'] = 13.2
    if df.loc[index,'FHR aver HDL, points'] == 13.0:
        df.loc[index,'FHR aver HDL'] = 15.6
    if df.loc[index,'FHR aver HDL, points'] == 14.0:
        df.loc[index,'FHR aver HDL'] = 18.4
    if df.loc[index,'FHR aver HDL, points'] == 15.0:
        df.loc[index,'FHR aver HDL'] = 21.6
    if df.loc[index,'FHR aver HDL, points'] == 16.0:
        df.loc[index,'FHR aver HDL'] = 25.3
    if df.loc[index,'FHR aver HDL, points'] == 17.0:
        df.loc[index,'FHR aver HDL'] = 29.4
    if df.loc[index,'FHR aver HDL, points'] >= 18.0:
        df.loc[index,'FHR aver HDL'] = 30.0
    # Recalculate to heart age
    if df.loc[index,'FHR aver HDL, points'] <= 0.0:
        df.loc[index,'FHR aver HDL Heart Age'] = 30.0
    if df.loc[index,'FHR aver HDL, points'] == 1.0:
        df.loc[index,'FHR aver HDL Heart Age'] = 32.0
    if df.loc[index,'FHR aver HDL, points'] == 2.0:
        df.loc[index,'FHR aver HDL Heart Age'] = 34.0
    if df.loc[index,'FHR aver HDL, points'] == 3.0:
        df.loc[index,'FHR aver HDL Heart Age'] = 36.0
    if df.loc[index,'FHR aver HDL, points'] == 4.0:
        df.loc[index,'FHR aver HDL Heart Age'] = 38.0
    if df.loc[index,'FHR aver HDL, points'] == 5.0:
        df.loc[index,'FHR aver HDL Heart Age'] = 40.0
    if df.loc[index,'FHR aver HDL, points'] == 6.0:
        df.loc[index,'FHR aver HDL Heart Age'] = 42.0
    if df.loc[index,'FHR aver HDL, points'] == 7.0:
        df.loc[index,'FHR aver HDL Heart Age'] = 45.0
    if df.loc[index,'FHR aver HDL, points'] == 8.0:
        df.loc[index,'FHR aver HDL Heart Age'] = 48.0
    if df.loc[index,'FHR aver HDL, points'] == 9.0:
        df.loc[index,'FHR aver HDL Heart Age'] = 51.0
    if df.loc[index,'FHR aver HDL, points'] == 10.0:
        df.loc[index,'FHR aver HDL Heart Age'] = 54.0
    if df.loc[index,'FHR aver HDL, points'] == 11.0:
        df.loc[index,'FHR aver HDL Heart Age'] = 57.0
    if df.loc[index,'FHR aver HDL, points'] == 12.0:
        df.loc[index,'FHR aver HDL Heart Age'] = 60.0
    if df.loc[index,'FHR aver HDL, points'] == 13.0:
        df.loc[index,'FHR aver HDL Heart Age'] = 64.0
    if df.loc[index,'FHR aver HDL, points'] == 14.0:
        df.loc[index,'FHR aver HDL Heart Age'] = 68.0
    if df.loc[index,'FHR aver HDL, points'] == 15.0:
        df.loc[index,'FHR aver HDL Heart Age'] = 72.0
    if df.loc[index,'FHR aver HDL, points'] == 16.0:
        df.loc[index,'FHR aver HDL Heart Age'] = 76.0
    if df.loc[index,'FHR aver HDL, points'] >= 17.0:
        df.loc[index,'FHR aver HDL Heart Age'] = 80.0

df_res_score = df[['FHR aver HDL', 'FHR aver HDL Heart Age']].copy()

In [ ]:
threshold = 10.0
df_res_score[f'FHR aver HDL group thr={threshold}'] = df_res_score['FHR aver HDL']
df_res_score[f'FHR aver HDL group thr={threshold}'].mask(df_res_score['FHR aver HDL'] >= threshold, 'High', inplace=True)
df_res_score[f'FHR aver HDL group thr={threshold}'].mask(df_res_score['FHR aver HDL'] < threshold, 'Low', inplace=True)

In [ ]:
df_res_score['FHR aver HDL Heart Age Acceleration'] = df_res_score['FHR aver HDL Heart Age'] - df['Age']
threshold = 0.0
df_res_score[f'FHR aver HDL Heart Age Acceleration group thr={threshold}'] = df_res_score['FHR aver HDL Heart Age Acceleration']
df_res_score[f'FHR aver HDL Heart Age Acceleration group thr={threshold}'].mask(df_res_score['FHR aver HDL Heart Age Acceleration'] >= threshold, 'High', inplace=True)
df_res_score[f'FHR aver HDL Heart Age Acceleration group thr={threshold}'].mask(df_res_score['FHR aver HDL Heart Age Acceleration'] < threshold, 'Low', inplace=True)

In [ ]:
df_scores = pd.read_excel(f"{path}/05_scores/df_scores.xlsx", index_col=0)
df_scores.index = df_scores.index.astype(str)
df_scores = pd.concat([df_scores, df_res_score], axis=1)
df_scores.to_excel(f'{path_save}/df_scores.xlsx')

## ASCVD Risk

### Without HDL cholesterol

In [ ]:
cols_for_score = ["терапевт - артериальное давление верхнее",
                  "биохимический анализ крови - холестерин",
                  "smoking",
                  "hypertension",
                  "Age"
]
for col in cols_for_score:
    df[col] = df[col].replace('нет', np.nan)
    df[col] = df[col].replace('норма', np.nan)
    df[col] = pd.to_numeric(df[col])

lincomb = -11.679980 + \
    0.064200 * df["Age"] + \
    0.482835 * 0.0 - \
    0.000061 * df["терапевт - артериальное давление верхнее"] * df["терапевт - артериальное давление верхнее"] + \
    0.038950 * df["терапевт - артериальное давление верхнее"] + \
    2.055533 * df["hypertension"] + \
    0.842209 * 0.0 + \
    0.895589 * df["smoking"] + \
    0.193307 * 0.0 - \
    0.014207 * df["терапевт - артериальное давление верхнее"] * df["hypertension"] + \
    0.011609 * df["терапевт - артериальное давление верхнее"] * 0.0 - \
    0.119460 * 0.0 * df["hypertension"] + \
    0.000025 * df["Age"] * df["терапевт - артериальное давление верхнее"] - \
    0.077214 * 0.0 * 0.0 - \
    0.226771 * 0.0 * df["smoking"] - \
    0.117749 * 0.0 * 0.0 + \
    0.004190 * df["терапевт - артериальное давление верхнее"] * 0.0 * df["hypertension"] - \
    0.000199 * df["Age"] * df["терапевт - артериальное давление верхнее"] * 0.0

risk_10 = 100.0 / (1.0 + np.exp(-lincomb))

df['ASCVD wo HDL'] = risk_10
df_res_score = df[['ASCVD wo HDL']].copy()

In [ ]:
threshold = 1.0
df_res_score[f'ASCVD wo HDL group thr={threshold}'] = df_res_score['ASCVD wo HDL']
df_res_score[f'ASCVD wo HDL group thr={threshold}'].mask(df_res_score['ASCVD wo HDL'] >= threshold, 'High', inplace=True)
df_res_score[f'ASCVD wo HDL group thr={threshold}'].mask(df_res_score['ASCVD wo HDL'] < threshold, 'Low', inplace=True)

In [ ]:
df_scores = pd.read_excel(f"{path}/05_scores/df_scores.xlsx", index_col=0)
df_scores.index = df_scores.index.astype(str)
df_scores = pd.concat([df_scores, df_res_score], axis=1)
df_scores.to_excel(f'{path_save}/df_scores.xlsx')

### With population-average HDL cholesterol

In [ ]:
cols_for_score = ["терапевт - артериальное давление верхнее",
                  "биохимический анализ крови - холестерин",
                  "smoking",
                  "hypertension",
                  "Age"
]
for col in cols_for_score:
    df[col] = df[col].replace('нет', np.nan)
    df[col] = df[col].replace('норма', np.nan)
    df[col] = pd.to_numeric(df[col])

lincomb = -11.679980 + \
    0.064200 * df["Age"] + \
    0.482835 * 0.0 - \
    0.000061 * df["терапевт - артериальное давление верхнее"] * df["терапевт - артериальное давление верхнее"] + \
    0.038950 * df["терапевт - артериальное давление верхнее"] + \
    2.055533 * df["hypertension"] + \
    0.842209 * 0.0 + \
    0.895589 * df["smoking"] + \
    0.193307 * df["биохимический анализ крови - холестерин"] / 1.0 - \
    0.014207 * df["терапевт - артериальное давление верхнее"] * df["hypertension"] + \
    0.011609 * df["терапевт - артериальное давление верхнее"] * 0.0 - \
    0.119460 * 0.0 * df["hypertension"] + \
    0.000025 * df["Age"] * df["терапевт - артериальное давление верхнее"] - \
    0.077214 * 0.0 * 0.0 - \
    0.226771 * 0.0 * df["smoking"] - \
    0.117749 * df["биохимический анализ крови - холестерин"] / 1.0 * 0.0 + \
    0.004190 * df["терапевт - артериальное давление верхнее"] * 0.0 * df["hypertension"] - \
    0.000199 * df["Age"] * df["терапевт - артериальное давление верхнее"] * 0.0

risk_10 = 100.0 / (1.0 + np.exp(-lincomb))

df['ASCVD aver HDL'] = risk_10
df_res_score = df[['ASCVD aver HDL']].copy()

In [ ]:
threshold = 2.0
df_res_score[f'ASCVD aver HDL group thr={threshold}'] = df_res_score['ASCVD aver HDL']
df_res_score[f'ASCVD aver HDL group thr={threshold}'].mask(df_res_score['ASCVD aver HDL'] >= threshold, 'High', inplace=True)
df_res_score[f'ASCVD aver HDL group thr={threshold}'].mask(df_res_score['ASCVD aver HDL'] < threshold, 'Low', inplace=True)

In [ ]:
df_scores = pd.read_excel(f"{path}/05_scores/df_scores.xlsx", index_col=0)
df_scores.index = df_scores.index.astype(str)
df_scores = pd.concat([df_scores, df_res_score], axis=1)
df_scores.to_excel(f'{path_save}/df_scores.xlsx')

## Reynolds Risk Score

### Without HDL cholesterol

In [ ]:
cols_for_score = ["терапевт - артериальное давление верхнее",
                  "биохимический анализ крови - холестерин",
                  "smoking",
                  "С-реакт белок мг л",
                  "Age"
]
for col in cols_for_score:
    df[col] = df[col].replace('нет', np.nan)
    df[col] = df[col].replace('норма', np.nan)
    df[col] = pd.to_numeric(df[col])

lincomb = 4.385 * np.log(df["Age"]) + \
    2.607 * np.log(df["терапевт - артериальное давление верхнее"]) + \
    0.963 * np.log(df["биохимический анализ крови - холестерин"]) - \
    0.772 * 0.0 + \
    0.405 * df["smoking"] + \
    0.102 * np.log(df["С-реакт белок мг л"]) + \
    0.541 * 0.0    

risk_10 = (1.0 - np.power(0.8990, np.exp(lincomb - 33.097))) * 100.0

df['Reynolds wo HDL'] = risk_10
df_res_score = df[['Reynolds wo HDL']].copy()

In [ ]:
threshold = 2.0
df_res_score[f'Reynolds wo HDL group thr={threshold}'] = df_res_score['Reynolds wo HDL']
df_res_score[f'Reynolds wo HDL group thr={threshold}'].mask(df_res_score['Reynolds wo HDL'] >= threshold, 'High', inplace=True)
df_res_score[f'Reynolds wo HDL group thr={threshold}'].mask(df_res_score['Reynolds wo HDL'] < threshold, 'Low', inplace=True)

In [ ]:
df_scores = pd.read_excel(f"{path}/05_scores/df_scores.xlsx", index_col=0)
df_scores.index = df_scores.index.astype(str)
df_scores = pd.concat([df_scores, df_res_score], axis=1)
df_scores.to_excel(f'{path_save}/df_scores.xlsx')

### With population-average HDL-cholesterol

In [ ]:
cols_for_score = ["терапевт - артериальное давление верхнее",
                  "биохимический анализ крови - холестерин",
                  "smoking",
                  "С-реакт белок мг л",
                  "Age"
]
for col in cols_for_score:
    df[col] = df[col].replace('нет', np.nan)
    df[col] = df[col].replace('норма', np.nan)
    df[col] = pd.to_numeric(df[col])

lincomb = 4.385 * np.log(df["Age"]) + \
    2.607 * np.log(df["терапевт - артериальное давление верхнее"]) + \
    0.963 * np.log(df["биохимический анализ крови - холестерин"]) - \
    0.772 * np.log(1.0) + \
    0.405 * df["smoking"] + \
    0.102 * np.log(df["С-реакт белок мг л"]) + \
    0.541 * 0.0    

risk_10 = (1.0 - np.power(0.8990, np.exp(lincomb - 33.097))) * 100.0

df['Reynolds aver HDL'] = risk_10
df_res_score = df[['Reynolds aver HDL']].copy()

In [ ]:
threshold = 2.0
df_res_score[f'Reynolds aver HDL group thr={threshold}'] = df_res_score['Reynolds aver HDL']
df_res_score[f'Reynolds aver HDL group thr={threshold}'].mask(df_res_score['Reynolds aver HDL'] >= threshold, 'High', inplace=True)
df_res_score[f'Reynolds aver HDL group thr={threshold}'].mask(df_res_score['Reynolds aver HDL'] < threshold, 'Low', inplace=True)

In [ ]:
df_scores = pd.read_excel(f"{path}/05_scores/df_scores.xlsx", index_col=0)
df_scores.index = df_scores.index.astype(str)
df_scores = pd.concat([df_scores, df_res_score], axis=1)
df_scores.to_excel(f'{path_save}/df_scores.xlsx')

## CKD-EPI

In [3]:
cols_for_score = ["биохимический анализ крови - креатинин",
                  "Age"
]
for col in cols_for_score:
    df[col] = df[col].replace('нет', np.nan)
    df[col] = df[col].replace('норма', np.nan)
    df[col] = pd.to_numeric(df[col])

# Креатинин сначала нужно перевести из ммоль/л в мг/дл (коэффициент 88.4)
gfr = 142.0 * \
    np.power(np.minimum((df["биохимический анализ крови - креатинин"] / 88.4) / 0.9, 1.0), -0.302) * \
    np.power(np.maximum((df["биохимический анализ крови - креатинин"] / 88.4) / 0.9, 1.0), -1.200) * \
    np.power(0.9938, df["Age"])

df['CKD-EPI'] = gfr
df_res_score = df[['CKD-EPI']].copy()

C:\Users\alena\AppData\Local\Temp\ipykernel_38988\2166584034.py:5: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df[col] = df[col].replace('нет', np.nan)


In [4]:
threshold = 75.0
df_res_score[f'CKD-EPI group thr={threshold}'] = df_res_score['CKD-EPI']
df_res_score[f'CKD-EPI group thr={threshold}'].mask(df_res_score['CKD-EPI'] >= threshold, 'Low', inplace=True)
df_res_score[f'CKD-EPI group thr={threshold}'].mask(df_res_score['CKD-EPI'] < threshold, 'High', inplace=True)

C:\Users\alena\AppData\Local\Temp\ipykernel_38988\492967097.py:3: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df_res_score[f'CKD-EPI group thr={threshold}'].mask(df_res_score['CKD-EPI'] >= threshold, 'Low', inplace=True)
C:\Users\alena\AppData\Local\Temp\ipykernel_38988\492967097.py:3: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'Low' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df_res_score[f'CKD

In [5]:
df_scores = pd.read_excel(f"{path}/05_scores/df_scores.xlsx", index_col=0)
df_scores.index = df_scores.index.astype(str)
df_scores = pd.concat([df_scores, df_res_score], axis=1)
df_scores.to_excel(f'{path_save}/df_scores.xlsx')

# Epigenetic clocks

## Load data

In [6]:
path = f"E:/YandexDisk/bbd/fmba"
path_pyaging = "E:/YandexDisk/pydnameth/datasets/pyaging"

epi_ages = []
epi_metrics = []

df_pheno = pd.read_excel(f"{path}/pheno_fixed.xlsx", index_col=0)
df_pheno.index = df_pheno.index.astype(str)
cols_pheno = ['дата рождения']

df_pyaging = pd.read_excel(f"{path}/dnam/processed/pheno.xlsx", index_col=0)
df_pyaging.index = df_pyaging.index.astype(str)
pyaging_meta = pd.read_excel(f"{path_pyaging}/clocks_meta.xlsx", index_col='Clock Name')
pyaging_meta['Clock Name'] = pyaging_meta.index
df_pyaging.rename(columns=dict(zip(pyaging_meta['Model ID'].values, pyaging_meta['Clock Name'].values)), inplace=True)
pyaging_meta.drop(index=['Knight', 'LeeControl', 'LeeRefinedRobust', 'LeeRobust', 'PedBE', 'RepliTali', 'ENCen100'], inplace=True)
epi_ages += pyaging_meta[pyaging_meta['Type'] == 'Age'].index.to_list()
epi_metrics += pyaging_meta[pyaging_meta['Type'] != 'Age'].index.to_list()
df_pyaging.rename(columns={'Age': 'Chronological Age (Epigenetics)'}, inplace=True)
cols_pyaging = ['Chronological Age (Epigenetics)', 'Sex', 'Tissue'] + pyaging_meta[pyaging_meta['Type'] == 'Age'].index.to_list() + pyaging_meta[pyaging_meta['Type'] != 'Age'].index.to_list()

df_scores = pd.read_excel(f"{path}/05_scores/df_scores.xlsx", index_col=0)
df_scores.index = df_scores.index.astype(str)
cols_scores = [#'SCORE', 'SCORE group thr=0.02', 'SCORE2 wo HDL', 'SCORE2 wo HDL group thr=0.05', 'SCORE2 aver HDL', 'SCORE2 aver HDL group thr=0.05', 
               #'FHR wo HDL', 'FHR wo HDL group thr=10.0', 'FHR wo HDL Heart Age Acceleration', 'FHR wo HDL Heart Age Acceleration group thr=0.0',
               #'FHR aver HDL', 'FHR aver HDL group thr=10.0', 'FHR aver HDL Heart Age Acceleration', 'FHR aver HDL Heart Age Acceleration group thr=0.0',
               #'ASCVD wo HDL', 'ASCVD wo HDL group thr=1.0', 'ASCVD aver HDL', 'ASCVD aver HDL group thr=2.0',
               #'Reynolds wo HDL', 'Reynolds wo HDL group thr=2.0', 'Reynolds aver HDL', 'Reynolds aver HDL group thr=2.0',
               'CKD-EPI', 'CKD-EPI group thr=75.0']

dfs = [df_pheno[cols_pheno], df_pyaging[cols_pyaging], df_scores[cols_scores]]
df_epi = reduce(lambda left,right: pd.merge(left, right, left_index=True, right_index=True), dfs)
df_epi['дата рождения'] = pd.to_datetime(df_epi['дата рождения'])
df_epi['Age'] = (pd.to_datetime("2024-11-11") - df_epi['дата рождения']) / np.timedelta64(1, 'D') / 365.25
df_epi = df_epi.drop(['3278', '4436'])

pheno_associations = {
    #'SCORE group thr=0.02': {
    #    'groups': ['Low', 'High'],
    #    'base': 'Low',
    #    'colors': {'Low': 'dodgerblue', 'High': 'crimson'}
    #},
    #'SCORE2 wo HDL group thr=0.05': {
    #    'groups': ['Low', 'High'],
    #    'base': 'Low',
    #    'colors': {'Low': 'dodgerblue', 'High': 'crimson'}
    #},
    #'SCORE2 aver HDL group thr=0.05': {
    #    'groups': ['Low', 'High'],
    #    'base': 'Low',
    #    'colors': {'Low': 'dodgerblue', 'High': 'crimson'}
    #},
    #'FHR wo HDL group thr=10.0': {
    #    'groups': ['Low', 'High'],
    #    'base': 'Low',
    #    'colors': {'Low': 'dodgerblue', 'High': 'crimson'}
    #},
    #'FHR wo HDL Heart Age Acceleration group thr=0.0': {
    #    'groups': ['Low', 'High'],
    #    'base': 'Low',
    #    'colors': {'Low': 'dodgerblue', 'High': 'crimson'}
    #},
    #'FHR aver HDL group thr=10.0': {
    #    'groups': ['Low', 'High'],
    #    'base': 'Low',
    #    'colors': {'Low': 'dodgerblue', 'High': 'crimson'}
    #},
    #'FHR aver HDL Heart Age Acceleration group thr=0.0': {
    #    'groups': ['Low', 'High'],
    #    'base': 'Low',
    #    'colors': {'Low': 'dodgerblue', 'High': 'crimson'}
    #},
    #'ASCVD wo HDL group thr=1.0': {
    #    'groups': ['Low', 'High'],
    #    'base': 'Low',
    #    'colors': {'Low': 'dodgerblue', 'High': 'crimson'}
    #},
    #'ASCVD aver HDL group thr=2.0': {
    #    'groups': ['Low', 'High'],
    #    'base': 'Low',
    #    'colors': {'Low': 'dodgerblue', 'High': 'crimson'}
    #},
    #'Reynolds wo HDL group thr=2.0': {
    #    'groups': ['Low', 'High'],
    #    'base': 'Low',
    #    'colors': {'Low': 'dodgerblue', 'High': 'crimson'}
    #},
    #'Reynolds aver HDL group thr=2.0': {
    #    'groups': ['Low', 'High'],
    #    'base': 'Low',
    #    'colors': {'Low': 'dodgerblue', 'High': 'crimson'}
    #},
    'CKD-EPI group thr=75.0': {
        'groups': ['Low', 'High'],
        'base': 'Low',
        'colors': {'Low': 'dodgerblue', 'High': 'crimson'}
    },
}

## Epigenetic ages (categorical score)

In [7]:
# Here we take all samples for linreg
for curr_score in pheno_associations:
    curr_score_name = curr_score.split(' group')[0]
    pathlib.Path(f"{path}/05_scores/{curr_score_name}/categorical/dnam").mkdir(parents=True, exist_ok=True)
    df_epi_scores = df_epi.loc[df_epi[curr_score].isin(pheno_associations[curr_score]['groups'])]
    df_stat = pd.DataFrame(index=epi_ages)
    score_vals = pheno_associations[curr_score]['groups']
    score_val_base = pheno_associations[curr_score]['base']
    score_colors = pheno_associations[curr_score]['colors']

    for epiage_id, epiage in enumerate(epi_ages):

        pathlib.Path(f"{path}/05_scores/{curr_score_name}/categorical/dnam/all").mkdir(parents=True, exist_ok=True)
        
        linreg_cx = smf.ols(formula=f"{epiage} ~ Age", data=df_epi_scores).fit()
        df_epi_scores[f"{epiage}_linear_pred_cx"] = linreg_cx.predict(df_epi_scores)
        df_epi_scores[f"{epiage}_acceleration_cx"] = df_epi_scores[epiage] - df_epi_scores[f"{epiage}_linear_pred_cx"]
        df_epi_scores[f"{epiage}"] = df_epi_scores["Age"] + df_epi_scores[f"{epiage}_acceleration_cx"]
        
        linreg = smf.ols(formula=f"{epiage} ~ Age", data=df_epi_scores.loc[:, :]).fit()
        df_epi_scores[f"{epiage}_linear_pred"] = linreg.predict(df_epi_scores)
        df_epi_scores[f"{epiage} acceleration"] = df_epi_scores[epiage] - df_epi_scores[f"{epiage}_linear_pred"]
        vals = {}
        for group in score_vals:
            vals[group] = df_epi_scores.loc[df_epi_scores[curr_score] == group, f"{epiage} acceleration"].values
            df_stat.at[epiage, f"Mean {group}"] = np.mean(vals[group])
            df_stat.at[epiage, f"Median {group}"] = np.median(vals[group])
            df_stat.at[epiage, f"Q75 {group}"], df_stat.at[epiage, f"Q25 {group}"] = np.percentile(vals[group], [75 , 25])
            df_stat.at[epiage, f"IQR {group}"] = df_stat.at[epiage, f"Q75 {group}"] - df_stat.at[epiage, f"Q25 {group}"]
            df_stat.at[epiage, f"Variation {group}"] = variation(vals[group])
        _, df_stat.at[epiage, "mannwhitneyu_pval"] = mannwhitneyu(vals[score_vals[0]], vals[score_vals[1]], alternative='two-sided')
        _, df_stat.at[epiage, "levene_pval"] = levene(vals[score_vals[0]], vals[score_vals[1]])
        regcov = smf.ols(formula=f"{epiage} ~ Q('{curr_score}') + Age", data=df_epi_scores).fit()
        reg_sum = regcov.summary2().tables[1]
        pvals_cols = reg_sum.index[reg_sum.index.str.contains(curr_score_name)].values
        for pval_col_id, pval_col in enumerate(pvals_cols):
            df_stat.at[epiage, f"ancova_{pval_col}_pval"] = reg_sum.at[pval_col, 'P>|t|']
        
        _, df_stat.at[epiage, "pearson_pval"] = pearsonr(df_epi_scores['Age'].values, df_epi_scores[epiage].values)
        _, df_stat.at[epiage, "spearman_pval"] = spearmanr(df_epi_scores['Age'].values, df_epi_scores[epiage].values)

    _, df_stat.loc[epi_ages, "mannwhitneyu_pval_fdr_bh"], _, _ = multipletests(df_stat.loc[epi_ages, "mannwhitneyu_pval"].values, 0.05, method='fdr_bh')
    _, df_stat.loc[epi_ages, "mannwhitneyu_pval_bonferroni"], _, _ = multipletests(df_stat.loc[epi_ages, "mannwhitneyu_pval"].values, 0.05, method='bonferroni')
    _, df_stat.loc[epi_ages, "mannwhitneyu_pval_simes-hochberg"], _, _ = multipletests(df_stat.loc[epi_ages, "mannwhitneyu_pval"].values, 0.05, method='simes-hochberg')
    _, df_stat.loc[epi_ages, "levene_pval_fdr_bh"], _, _ = multipletests(df_stat.loc[epi_ages, "levene_pval"].values, 0.05, method='fdr_bh')
    _, df_stat.loc[epi_ages, "levene_pval_bonferroni"], _, _ = multipletests(df_stat.loc[epi_ages, "levene_pval"].values, 0.05, method='bonferroni')
    _, df_stat.loc[epi_ages, "levene_pval_simes-hochberg"], _, _ = multipletests(df_stat.loc[epi_ages, "levene_pval"].values, 0.05, method='simes-hochberg')
    
    pvals_cols_ancova = df_stat.columns[df_stat.columns.str.contains(curr_score_name)].values
    for pval_col in pvals_cols_ancova:
        _, df_stat.loc[epi_ages, f"{pval_col}_fdr_bh"], _, _ = multipletests(df_stat.loc[epi_ages, pval_col].values, 0.05, method='fdr_bh')
        _, df_stat.loc[epi_ages, f"{pval_col}_bonferroni"], _, _ = multipletests(df_stat.loc[epi_ages, pval_col].values, 0.05, method='bonferroni')
        _, df_stat.loc[epi_ages, f"{pval_col}_simes-hochberg"], _, _ = multipletests(df_stat.loc[epi_ages, pval_col].values, 0.05, method='simes-hochberg')
    
    _, df_stat.loc[epi_ages, "pearson_pval_fdr_bh"], _, _ = multipletests(df_stat.loc[epi_ages, "pearson_pval"].values, 0.05, method='fdr_bh')
    _, df_stat.loc[epi_ages, "pearson_pval_bonferroni"], _, _ = multipletests(df_stat.loc[epi_ages, "pearson_pval"].values, 0.05, method='bonferroni')
    _, df_stat.loc[epi_ages, "pearson_pval_simes-hochberg"], _, _ = multipletests(df_stat.loc[epi_ages, "pearson_pval"].values, 0.05, method='simes-hochberg')
    _, df_stat.loc[epi_ages, "spearman_pval_fdr_bh"], _, _ = multipletests(df_stat.loc[epi_ages, "spearman_pval"].values, 0.05, method='fdr_bh')
    _, df_stat.loc[epi_ages, "spearman_pval_bonferroni"], _, _ = multipletests(df_stat.loc[epi_ages, "spearman_pval"].values, 0.05, method='bonferroni')
    _, df_stat.loc[epi_ages, "spearman_pval_simes-hochberg"], _, _ = multipletests(df_stat.loc[epi_ages, "spearman_pval"].values, 0.05, method='simes-hochberg')

    df_stat.sort_values([f"mannwhitneyu_pval"], ascending=[True], inplace=True)
    df_stat.to_excel(f"{path}/05_scores/{curr_score_name}/categorical/dnam/all/ages_corrected.xlsx")
    
    hist_bins = np.linspace(5, 115, 23)
    fig, ax = plt.subplots(figsize=(6, 4))
    histplot = sns.histplot(
        data=df_epi_scores,
        bins=hist_bins,
        edgecolor='k',
        linewidth=1,
        x="Age",
        hue=curr_score,
        palette=score_colors,
        ax=ax
    )
    histplot.set(xlim=(0, 120))
    plt.savefig(f"{path}/05_scores/{curr_score_name}/categorical/dnam/all/hist_age_corrected.png", bbox_inches='tight', dpi=200)
    plt.savefig(f"{path}/05_scores/{curr_score_name}/categorical/dnam/all/hist_age_corrected.pdf", bbox_inches='tight')
    plt.close(fig)
    
    for stat_test in [x.replace('_pval', '') for x in pvals_cols_ancova] + ['mannwhitneyu', 'levene']:
        df_fig = df_stat.copy()
        df_fig.sort_values([f"{stat_test}_pval"], ascending=[True], inplace=True)
        df_fig['Features'] = df_fig.index
        df_fig[f'{stat_test}_pval_fdr_bh_log'] = -np.log10(df_fig[f'{stat_test}_pval_fdr_bh'])
        df_fig['color'] = 'pink'
        df_fig.loc[df_fig[f'{stat_test}_pval_fdr_bh'] < 0.05, 'color'] = 'red'
        sns.set_theme(style='ticks')
        fig, ax = plt.subplots(figsize=(3, df_fig.shape[0] * 0.5))
        barplot = sns.barplot(
            data=df_fig,
            y='Features',
            x=f'{stat_test}_pval_fdr_bh_log',
            edgecolor='black',
            palette=df_fig['color'].values,
            ax=ax,
        )
        ax.set_xlabel(r"$-\log_{10}(\mathrm{p-value})$")
        ax.xaxis.tick_top()
        ax.xaxis.set_label_position('top')
        ax.set_ylabel('')
        plt.savefig(f"{path}/05_scores/{curr_score_name}/categorical/dnam/all/ages_pvals_{stat_test}_corrected.png", bbox_inches='tight', dpi=200)
        plt.savefig(f"{path}/05_scores/{curr_score_name}/categorical/dnam/all/ages_pvals_{stat_test}_corrected.pdf", bbox_inches='tight')
        plt.close(fig)
        
    sns.set_theme(style='ticks')
    fig = plt.figure(
        figsize=(24, 28),
    )
    subfigs = fig.subfigures(
        nrows=7,
        ncols=4,
    )
    for epiage_id, epiage in enumerate(df_stat.index.values):
        row_id, col_id = divmod(epiage_id, 4)

        axs = subfigs[row_id, col_id].subplot_mosaic(
            [
                ['11', '12'],
                ['21', '22'],
            ],
            height_ratios=[1, 4],
            width_ratios=[3, 1.5],
            gridspec_kw={
                "bottom": 0.14,
                "top": 0.95,
                "wspace": 0.37,
                "hspace": 0.01,
            },
        )
        
        ds_table = pd.DataFrame(index=[fr"Pearson $\rho$", "Pearson p-value", fr"Spearman $\rho$", "Spearman p-value"], columns=[epiage])
        rho, _ = pearsonr(df_epi_scores['Age'].values, df_epi_scores[epiage].values)
        sp_rho, _ = spearmanr(df_epi_scores['Age'].values, df_epi_scores[epiage].values)
        pearson_pval = df_stat.at[epiage, "pearson_pval_fdr_bh"]
        spearman_pval = df_stat.at[epiage, "spearman_pval_fdr_bh"]
        ds_table.at[fr"Pearson $\rho$", epiage] = f"{rho:0.2f}"
        ds_table.at["Pearson p-value", epiage] = f"{pearson_pval:.2e}"
        ds_table.at[fr"Spearman $\rho$", epiage] = f"{sp_rho:0.2f}"
        ds_table.at["Spearman p-value", epiage] = f"{spearman_pval:.2e}"
        col_defs = [
            ColumnDefinition(
                name="index",
                title=epiage,
                textprops={"ha": "left"},
                width=4.5,
            ),
            ColumnDefinition(
                name=epiage,
                title='',
                textprops={"ha": "center"},
                width=2.0,
            ),
        ]
        table = Table(
            ds_table,
            column_definitions=col_defs,
            row_dividers=True,
            footer_divider=False,
            ax=axs['11'],
            textprops={"fontsize": 7},
            row_divider_kw={"linewidth": 1, "linestyle": (0, (1, 1))},
            col_label_divider_kw={"linewidth": 1, "linestyle": "-"},
            column_border_kw={"linewidth": 1, "linestyle": "-"},
        ).autoset_fontcolors(colnames=[epiage])
        
        axs['12'].axis('off')
        
        xy_min = df_epi_scores[['Age', epiage]].min().min()
        xy_max = df_epi_scores[['Age', epiage]].max().max()
        xy_ptp = xy_max - xy_min
        bisect = sns.lineplot(
            x=[xy_min - 0.1 * xy_ptp, xy_max + 0.1 * xy_ptp],
            y=[xy_min - 0.1 * xy_ptp, xy_max + 0.1 * xy_ptp],
            linestyle='--',
            color='black',
            linewidth=1.0,
            ax=axs['21']
        )
        regplot = sns.regplot(
            data=df_epi_scores.loc[:, :],
            x='Age',
            y=epiage,
            color='dimgray',
            scatter=False,
            truncate=False,
            ax=axs['21']
        )
        scatter = sns.scatterplot(
            data=df_epi_scores,
            x='Age',
            y=epiage,
            hue=curr_score,
            palette=score_colors,
            linewidth=0.5,
            alpha=0.75,
            edgecolor="k",
            s=20,
            hue_order=list(score_colors.keys()),
            legend=True,
            ax=axs['21'],
        )
        axs['21'].set_xlim(xy_min - 0.1 * xy_ptp, xy_max + 0.1 * xy_ptp)
        axs['21'].set_ylim(xy_min - 0.1 * xy_ptp, xy_max + 0.1 * xy_ptp)
        scatter.legend_.set_title(curr_score_name)
        
        sns.violinplot(
            data=df_epi_scores,
            x=curr_score,
            y=f"{epiage} acceleration",
            hue=curr_score,
            palette=score_colors,
            density_norm='width',
            order=score_vals,
            saturation=0.75,
            linewidth=1.0,
            ax=axs['22'],
            legend=False,
            cut=0,
        )
        axs['22'].set_ylabel(f"{epiage} acceleration")
        axs['22'].set_xlabel(curr_score_name)
        mannwhitneyu_pval = df_stat.at[epiage, "mannwhitneyu_pval_fdr_bh"]
        levene_pval = df_stat.at[epiage, "levene_pval_fdr_bh"]
        title = f'Mann-Whitney: {mannwhitneyu_pval:.2e}\nLevene: {levene_pval:.2e}'
        for pval_col in pvals_cols_ancova:
            title += f"\nANCOVA: {df_stat.at[epiage, pval_col + '_fdr_bh']:.2e}"
        axs['22'].set_title(title)

    fig.savefig(f"{path}/05_scores/{curr_score_name}/categorical/dnam/all/ages_distribution_corrected.png", bbox_inches='tight', dpi=200)
    fig.savefig(f"{path}/05_scores/{curr_score_name}/categorical/dnam/all/ages_distribution_corrected.pdf", bbox_inches='tight')
    plt.close(fig)

C:\Users\alena\AppData\Local\Temp\ipykernel_38988\80773705.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_epi_scores[f"{epiage}_linear_pred_cx"] = linreg_cx.predict(df_epi_scores)
C:\Users\alena\AppData\Local\Temp\ipykernel_38988\80773705.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_epi_scores[f"{epiage}_acceleration_cx"] = df_epi_scores[epiage] - df_epi_scores[f"{epiage}_linear_pred_cx"]
C:\Users\alena\AppData\Local\Temp\ipykernel_38988\80773705.py:18: SettingWithCopyWarning: 
A value i

In [8]:
# Here we take samples with low risk for linreg
for curr_score in pheno_associations:
    curr_score_name = curr_score.split(' group')[0]
    pathlib.Path(f"{path}/05_scores/{curr_score_name}/categorical/dnam").mkdir(parents=True, exist_ok=True)
    df_epi_scores = df_epi.loc[df_epi[curr_score].isin(pheno_associations[curr_score]['groups'])]
    df_stat = pd.DataFrame(index=epi_ages)
    score_vals = pheno_associations[curr_score]['groups']
    score_val_base = pheno_associations[curr_score]['base']
    score_colors = pheno_associations[curr_score]['colors']

    for epiage_id, epiage in enumerate(epi_ages):

        pathlib.Path(f"{path}/05_scores/{curr_score_name}/categorical/dnam/low_risk").mkdir(parents=True, exist_ok=True)
        
        linreg_cx = smf.ols(formula=f"{epiage} ~ Age", data=df_epi_scores).fit()
        df_epi_scores[f"{epiage}_linear_pred_cx"] = linreg_cx.predict(df_epi_scores)
        df_epi_scores[f"{epiage}_acceleration_cx"] = df_epi_scores[epiage] - df_epi_scores[f"{epiage}_linear_pred_cx"]
        df_epi_scores[f"{epiage}"] = df_epi_scores["Age"] + df_epi_scores[f"{epiage}_acceleration_cx"]
        
        linreg = smf.ols(formula=f"{epiage} ~ Age", data=df_epi_scores.loc[df_epi_scores[curr_score] == score_val_base, :]).fit()
        df_epi_scores[f"{epiage}_linear_pred"] = linreg.predict(df_epi_scores)
        df_epi_scores[f"{epiage} acceleration"] = df_epi_scores[epiage] - df_epi_scores[f"{epiage}_linear_pred"]
        vals = {}
        for group in score_vals:
            vals[group] = df_epi_scores.loc[df_epi_scores[curr_score] == group, f"{epiage} acceleration"].values
            df_stat.at[epiage, f"Mean {group}"] = np.mean(vals[group])
            df_stat.at[epiage, f"Median {group}"] = np.median(vals[group])
            df_stat.at[epiage, f"Q75 {group}"], df_stat.at[epiage, f"Q25 {group}"] = np.percentile(vals[group], [75 , 25])
            df_stat.at[epiage, f"IQR {group}"] = df_stat.at[epiage, f"Q75 {group}"] - df_stat.at[epiage, f"Q25 {group}"]
            df_stat.at[epiage, f"Variation {group}"] = variation(vals[group])
        _, df_stat.at[epiage, "mannwhitneyu_pval"] = mannwhitneyu(vals[score_vals[0]], vals[score_vals[1]], alternative='two-sided')
        _, df_stat.at[epiage, "levene_pval"] = levene(vals[score_vals[0]], vals[score_vals[1]])
        regcov = smf.ols(formula=f"{epiage} ~ Q('{curr_score}') + Age", data=df_epi_scores).fit()
        reg_sum = regcov.summary2().tables[1]
        pvals_cols = reg_sum.index[reg_sum.index.str.contains(curr_score_name)].values
        for pval_col_id, pval_col in enumerate(pvals_cols):
            df_stat.at[epiage, f"ancova_{pval_col}_pval"] = reg_sum.at[pval_col, 'P>|t|']
        
        _, df_stat.at[epiage, "pearson_pval"] = pearsonr(df_epi_scores.loc[df_epi_scores[curr_score] == score_val_base]['Age'].values, df_epi_scores.loc[df_epi_scores[curr_score] == score_val_base][epiage].values)
        _, df_stat.at[epiage, "spearman_pval"] = spearmanr(df_epi_scores.loc[df_epi_scores[curr_score] == score_val_base]['Age'].values, df_epi_scores.loc[df_epi_scores[curr_score] == score_val_base][epiage].values)

    _, df_stat.loc[epi_ages, "mannwhitneyu_pval_fdr_bh"], _, _ = multipletests(df_stat.loc[epi_ages, "mannwhitneyu_pval"].values, 0.05, method='fdr_bh')
    _, df_stat.loc[epi_ages, "mannwhitneyu_pval_bonferroni"], _, _ = multipletests(df_stat.loc[epi_ages, "mannwhitneyu_pval"].values, 0.05, method='bonferroni')
    _, df_stat.loc[epi_ages, "mannwhitneyu_pval_simes-hochberg"], _, _ = multipletests(df_stat.loc[epi_ages, "mannwhitneyu_pval"].values, 0.05, method='simes-hochberg')
    _, df_stat.loc[epi_ages, "levene_pval_fdr_bh"], _, _ = multipletests(df_stat.loc[epi_ages, "levene_pval"].values, 0.05, method='fdr_bh')
    _, df_stat.loc[epi_ages, "levene_pval_bonferroni"], _, _ = multipletests(df_stat.loc[epi_ages, "levene_pval"].values, 0.05, method='bonferroni')
    _, df_stat.loc[epi_ages, "levene_pval_simes-hochberg"], _, _ = multipletests(df_stat.loc[epi_ages, "levene_pval"].values, 0.05, method='simes-hochberg')
    pvals_cols_ancova = df_stat.columns[df_stat.columns.str.contains(curr_score_name)].values
    for pval_col in pvals_cols_ancova:
        _, df_stat.loc[epi_ages, f"{pval_col}_fdr_bh"], _, _ = multipletests(df_stat.loc[epi_ages, pval_col].values, 0.05, method='fdr_bh')
        _, df_stat.loc[epi_ages, f"{pval_col}_bonferroni"], _, _ = multipletests(df_stat.loc[epi_ages, pval_col].values, 0.05, method='bonferroni')
        _, df_stat.loc[epi_ages, f"{pval_col}_simes-hochberg"], _, _ = multipletests(df_stat.loc[epi_ages, pval_col].values, 0.05, method='simes-hochberg')

    _, df_stat.loc[epi_ages, "pearson_pval_fdr_bh"], _, _ = multipletests(df_stat.loc[epi_ages, "pearson_pval"].values, 0.05, method='fdr_bh')
    _, df_stat.loc[epi_ages, "pearson_pval_bonferroni"], _, _ = multipletests(df_stat.loc[epi_ages, "pearson_pval"].values, 0.05, method='bonferroni')
    _, df_stat.loc[epi_ages, "pearson_pval_simes-hochberg"], _, _ = multipletests(df_stat.loc[epi_ages, "pearson_pval"].values, 0.05, method='simes-hochberg')
    _, df_stat.loc[epi_ages, "spearman_pval_fdr_bh"], _, _ = multipletests(df_stat.loc[epi_ages, "spearman_pval"].values, 0.05, method='fdr_bh')
    _, df_stat.loc[epi_ages, "spearman_pval_bonferroni"], _, _ = multipletests(df_stat.loc[epi_ages, "spearman_pval"].values, 0.05, method='bonferroni')
    _, df_stat.loc[epi_ages, "spearman_pval_simes-hochberg"], _, _ = multipletests(df_stat.loc[epi_ages, "spearman_pval"].values, 0.05, method='simes-hochberg')
    
    df_stat.sort_values([f"mannwhitneyu_pval"], ascending=[True], inplace=True)
    df_stat.to_excel(f"{path}/05_scores/{curr_score_name}/categorical/dnam/low_risk/ages_corrected.xlsx")
    
    hist_bins = np.linspace(5, 115, 23)
    fig, ax = plt.subplots(figsize=(6, 4))
    histplot = sns.histplot(
        data=df_epi_scores,
        bins=hist_bins,
        edgecolor='k',
        linewidth=1,
        x="Age",
        hue=curr_score,
        palette=score_colors,
        ax=ax
    )
    histplot.set(xlim=(0, 120))
    plt.savefig(f"{path}/05_scores/{curr_score_name}/categorical/dnam/low_risk/hist_age_corrected.png", bbox_inches='tight', dpi=200)
    plt.savefig(f"{path}/05_scores/{curr_score_name}/categorical/dnam/low_risk/hist_age_corrected.pdf", bbox_inches='tight')
    plt.close(fig)
    
    for stat_test in [x.replace('_pval', '') for x in pvals_cols_ancova] + ['mannwhitneyu', 'levene']:
        df_fig = df_stat.copy()
        df_fig.sort_values([f"{stat_test}_pval"], ascending=[True], inplace=True)
        df_fig['Features'] = df_fig.index
        df_fig[f'{stat_test}_pval_fdr_bh_log'] = -np.log10(df_fig[f'{stat_test}_pval_fdr_bh'])
        df_fig['color'] = 'pink'
        df_fig.loc[df_fig[f'{stat_test}_pval_fdr_bh'] < 0.05, 'color'] = 'red'
        sns.set_theme(style='ticks')
        fig, ax = plt.subplots(figsize=(3, df_fig.shape[0] * 0.5))
        barplot = sns.barplot(
            data=df_fig,
            y='Features',
            x=f'{stat_test}_pval_fdr_bh_log',
            edgecolor='black',
            palette=df_fig['color'].values,
            ax=ax,
        )
        ax.set_xlabel(r"$-\log_{10}(\mathrm{p-value})$")
        ax.xaxis.tick_top()
        ax.xaxis.set_label_position('top')
        ax.set_ylabel('')
        plt.savefig(f"{path}/05_scores/{curr_score_name}/categorical/dnam/low_risk/ages_pvals_{stat_test}_corrected.png", bbox_inches='tight', dpi=200)
        plt.savefig(f"{path}/05_scores/{curr_score_name}/categorical/dnam/low_risk/ages_pvals_{stat_test}_corrected.pdf", bbox_inches='tight')
        plt.close(fig)
        
    sns.set_theme(style='ticks')
    fig = plt.figure(
        figsize=(24, 28),
    )
    subfigs = fig.subfigures(
        nrows=7,
        ncols=4,
    )
    for epiage_id, epiage in enumerate(df_stat.index.values):
        row_id, col_id = divmod(epiage_id, 4)

        axs = subfigs[row_id, col_id].subplot_mosaic(
            [
                ['11', '12'],
                ['21', '22'],
            ],
            height_ratios=[1, 4],
            width_ratios=[3, 1.5],
            gridspec_kw={
                "bottom": 0.14,
                "top": 0.95,
                "wspace": 0.43,
                "hspace": 0.01,
            },
        )
        
        ds_table = pd.DataFrame(index=[fr"Pearson $\rho$", "Pearson p-value", fr"Spearman $\rho$", "Spearman p-value"], columns=[epiage])
        rho, _ = pearsonr(df_epi_scores.loc[df_epi_scores[curr_score] == score_val_base]['Age'].values, df_epi_scores.loc[df_epi_scores[curr_score] == score_val_base][epiage].values)
        sp_rho, _ = spearmanr(df_epi_scores.loc[df_epi_scores[curr_score] == score_val_base]['Age'].values, df_epi_scores.loc[df_epi_scores[curr_score] == score_val_base][epiage].values)
        pearson_pval = df_stat.at[epiage, "pearson_pval_fdr_bh"]
        spearman_pval = df_stat.at[epiage, "spearman_pval_fdr_bh"]
        ds_table.at[fr"Pearson $\rho$", epiage] = f"{rho:0.2f}"
        ds_table.at["Pearson p-value", epiage] = f"{pearson_pval:.2e}"
        ds_table.at[fr"Spearman $\rho$", epiage] = f"{sp_rho:0.2f}"
        ds_table.at["Spearman p-value", epiage] = f"{spearman_pval:.2e}"
        col_defs = [
            ColumnDefinition(
                name="index",
                title=epiage,
                textprops={"ha": "left"},
                width=4.5,
            ),
            ColumnDefinition(
                name=epiage,
                title='',
                textprops={"ha": "center"},
                width=2.0,
            ),
        ]
        table = Table(
            ds_table,
            column_definitions=col_defs,
            row_dividers=True,
            footer_divider=False,
            ax=axs['11'],
            textprops={"fontsize": 7},
            row_divider_kw={"linewidth": 1, "linestyle": (0, (1, 1))},
            col_label_divider_kw={"linewidth": 1, "linestyle": "-"},
            column_border_kw={"linewidth": 1, "linestyle": "-"},
        ).autoset_fontcolors(colnames=[epiage])
        
        axs['12'].axis('off')
        
        xy_min = df_epi_scores[['Age', epiage]].min().min()
        xy_max = df_epi_scores[['Age', epiage]].max().max()
        xy_ptp = xy_max - xy_min
        bisect = sns.lineplot(
            x=[xy_min - 0.1 * xy_ptp, xy_max + 0.1 * xy_ptp],
            y=[xy_min - 0.1 * xy_ptp, xy_max + 0.1 * xy_ptp],
            linestyle='--',
            color='black',
            linewidth=1.0,
            ax=axs['21']
        )
        regplot = sns.regplot(
            data=df_epi_scores.loc[df_epi_scores[curr_score] == score_val_base, :],
            x='Age',
            y=epiage,
            color=score_colors[score_val_base],
            scatter=False,
            truncate=False,
            ax=axs['21']
        )
        scatter = sns.scatterplot(
            data=df_epi_scores,
            x='Age',
            y=epiage,
            hue=curr_score,
            palette=score_colors,
            linewidth=0.5,
            alpha=0.75,
            edgecolor="k",
            s=20,
            hue_order=list(score_colors.keys()),
            legend=True,
            ax=axs['21'],
        )
        axs['21'].set_xlim(xy_min - 0.1 * xy_ptp, xy_max + 0.1 * xy_ptp)
        axs['21'].set_ylim(xy_min - 0.1 * xy_ptp, xy_max + 0.1 * xy_ptp)
        scatter.legend_.set_title(curr_score_name)
        
        sns.violinplot(
            data=df_epi_scores,
            x=curr_score,
            y=f"{epiage} acceleration",
            hue=curr_score,
            palette=score_colors,
            density_norm='width',
            order=score_vals,
            saturation=0.75,
            linewidth=1.0,
            ax=axs['22'],
            legend=False,
            cut=0,
        )
        axs['22'].set_ylabel(f"{epiage} acceleration")
        axs['22'].set_xlabel(curr_score_name)
        mannwhitneyu_pval = df_stat.at[epiage, "mannwhitneyu_pval_fdr_bh"]
        levene_pval = df_stat.at[epiage, "levene_pval_fdr_bh"]
        title = f'Mann-Whitney: {mannwhitneyu_pval:.2e}\nLevene: {levene_pval:.2e}'
        for pval_col in pvals_cols_ancova:
            title += f"\nANCOVA: {df_stat.at[epiage, pval_col + '_fdr_bh']:.2e}"
        axs['22'].set_title(title)

    fig.savefig(f"{path}/05_scores/{curr_score_name}/categorical/dnam/low_risk/ages_distribution_corrected.png", bbox_inches='tight', dpi=200)
    fig.savefig(f"{path}/05_scores/{curr_score_name}/categorical/dnam/low_risk/ages_distribution_corrected.pdf", bbox_inches='tight')
    plt.close(fig)

C:\Users\alena\AppData\Local\Temp\ipykernel_38988\180172624.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_epi_scores[f"{epiage}_linear_pred_cx"] = linreg_cx.predict(df_epi_scores)
C:\Users\alena\AppData\Local\Temp\ipykernel_38988\180172624.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_epi_scores[f"{epiage}_acceleration_cx"] = df_epi_scores[epiage] - df_epi_scores[f"{epiage}_linear_pred_cx"]
C:\Users\alena\AppData\Local\Temp\ipykernel_38988\180172624.py:18: SettingWithCopyWarning: 
A valu

## Epigenetic ages (continuous score)

In [9]:
# Here we take all samples for linreg
for curr_score in pheno_associations:
    curr_score_name = curr_score.split(' group')[0]
    pathlib.Path(f"{path}/05_scores/{curr_score_name}/continuous/dnam/all").mkdir(parents=True, exist_ok=True)
    df_epi_scores = df_epi.loc[df_epi[curr_score].isin(pheno_associations[curr_score]['groups'])]
    df_stat = pd.DataFrame(index=epi_ages)
    score_vals = pheno_associations[curr_score]['groups']
    score_val_base = pheno_associations[curr_score]['base']
    score_colors = pheno_associations[curr_score]['colors']

    for epiage_id, epiage in enumerate(epi_ages):
        
        linreg = smf.ols(formula=f"{epiage} ~ Age", data=df_epi_scores).fit()
        df_epi_scores[f"{epiage}_linear_pred"] = linreg.predict(df_epi_scores)
        df_epi_scores[f"{epiage} acceleration"] = df_epi_scores[epiage] - df_epi_scores[f"{epiage}_linear_pred"]
        vals = {}
        for group in score_vals:
            vals[group] = df_epi_scores.loc[df_epi_scores[curr_score] == group, f"{epiage} acceleration"].values
        _, df_stat.at[epiage, "mannwhitneyu_pval"] = mannwhitneyu(vals[score_vals[0]], vals[score_vals[1]], alternative='two-sided')
        _, df_stat.at[epiage, "levene_pval"] = levene(vals[score_vals[0]], vals[score_vals[1]])
        regcov = smf.ols(formula=f"{epiage} ~ Q('{curr_score}') + Age", data=df_epi_scores).fit()
        reg_sum = regcov.summary2().tables[1]
        pvals_cols = reg_sum.index[reg_sum.index.str.contains(curr_score_name)].values
        for pval_col_id, pval_col in enumerate(pvals_cols):
            df_stat.at[epiage, f"ancova_{pval_col}_pval"] = reg_sum.at[pval_col, 'P>|t|']
        
        _, df_stat.at[epiage, "pearson_pval"] = pearsonr(df_epi_scores[curr_score_name].values, df_epi_scores[f"{epiage} acceleration"].values)
        _, df_stat.at[epiage, "spearman_pval"] = spearmanr(df_epi_scores[curr_score_name].values, df_epi_scores[f"{epiage} acceleration"].values)

    _, df_stat.loc[epi_ages, "mannwhitneyu_pval_fdr_bh"], _, _ = multipletests(df_stat.loc[epi_ages, "mannwhitneyu_pval"].values, 0.05, method='fdr_bh')
    _, df_stat.loc[epi_ages, "levene_pval_fdr_bh"], _, _ = multipletests(df_stat.loc[epi_ages, "levene_pval"].values, 0.05, method='fdr_bh')
    pvals_cols_ancova = df_stat.columns[df_stat.columns.str.contains(curr_score_name)].values
    for pval_col in pvals_cols_ancova:
        _, df_stat.loc[epi_ages, f"{pval_col}_fdr_bh"], _, _ = multipletests(df_stat.loc[epi_ages, pval_col].values, 0.05, method='fdr_bh')
    
    _, df_stat.loc[epi_ages, "pearson_pval_fdr_bh"], _, _ = multipletests(df_stat.loc[epi_ages, "pearson_pval"].values, 0.05, method='fdr_bh')
    _, df_stat.loc[epi_ages, "spearman_pval_fdr_bh"], _, _ = multipletests(df_stat.loc[epi_ages, "spearman_pval"].values, 0.05, method='fdr_bh')
    
    hist_bins = np.linspace(5, 115, 23)
    fig, ax = plt.subplots(figsize=(6, 4))
    histplot = sns.histplot(
        data=df_epi_scores,
        bins=hist_bins,
        edgecolor='k',
        linewidth=1,
        x="Age",
        hue=curr_score,
        palette=score_colors,
        ax=ax
    )
    histplot.set(xlim=(0, 120))
    plt.savefig(f"{path}/05_scores/{curr_score_name}/continuous/dnam/all/hist_age_corrected.png", bbox_inches='tight', dpi=200)
    plt.savefig(f"{path}/05_scores/{curr_score_name}/continuous/dnam/all/hist_age_corrected.pdf", bbox_inches='tight')
    plt.close(fig)
        
    sns.set_theme(style='ticks')
    fig = plt.figure(
        figsize=(24, 28),
    )
    subfigs = fig.subfigures(
        nrows=7,
        ncols=4,
    )
    for epiage_id, epiage in enumerate(df_stat.index.values):
        row_id, col_id = divmod(epiage_id, 4)

        axs = subfigs[row_id, col_id].subplot_mosaic(
            [
                ['11', '12'],
                ['21', '22'],
            ],
            height_ratios=[1, 4],
            width_ratios=[3, 1.5],
            gridspec_kw={
                "bottom": 0.14,
                "top": 0.95,
                "wspace": 0.37,
                "hspace": 0.01,
            },
        )
        
        ds_table = pd.DataFrame(index=[fr"Pearson $\rho$", "Pearson p-value", fr"Spearman $\rho$", "Spearman p-value"], columns=[epiage])
        rho, _ = pearsonr(df_epi_scores[curr_score_name].values, df_epi_scores[f"{epiage} acceleration"].values)
        sp_rho, _ = spearmanr(df_epi_scores[curr_score_name].values, df_epi_scores[f"{epiage} acceleration"].values)
        pearson_pval = df_stat.at[epiage, "pearson_pval_fdr_bh"]
        spearman_pval = df_stat.at[epiage, "spearman_pval_fdr_bh"]
        ds_table.at[fr"Pearson $\rho$", epiage] = f"{rho:0.2f}"
        ds_table.at["Pearson p-value", epiage] = f"{pearson_pval:.2e}"
        ds_table.at[fr"Spearman $\rho$", epiage] = f"{sp_rho:0.2f}"
        ds_table.at["Spearman p-value", epiage] = f"{spearman_pval:.2e}"
        col_defs = [
            ColumnDefinition(
                name="index",
                title=epiage,
                textprops={"ha": "left"},
                width=4.5,
            ),
            ColumnDefinition(
                name=epiage,
                title='',
                textprops={"ha": "center"},
                width=2.0,
            ),
        ]
        table = Table(
            ds_table,
            column_definitions=col_defs,
            row_dividers=True,
            footer_divider=False,
            ax=axs['11'],
            textprops={"fontsize": 7},
            row_divider_kw={"linewidth": 1, "linestyle": (0, (1, 1))},
            col_label_divider_kw={"linewidth": 1, "linestyle": "-"},
            column_border_kw={"linewidth": 1, "linestyle": "-"},
        ).autoset_fontcolors(colnames=[epiage])
        
        axs['12'].axis('off')
        
        x_min = df_epi_scores[f"{epiage} acceleration"].min()
        x_max = df_epi_scores[f"{epiage} acceleration"].max()
        x_ptp = x_max - x_min
        y_min = df_epi_scores[curr_score_name].min()
        y_max = df_epi_scores[curr_score_name].max()
        y_ptp = y_max - y_min
        regplot = sns.regplot(
            data=df_epi_scores.loc[:, :],
            x=f'{epiage} acceleration',
            y=curr_score_name,
            color='dimgray',
            scatter=False,
            ax=axs['21']
        )
        scatter = sns.scatterplot(
            data=df_epi_scores,
            x=f"{epiage} acceleration",
            y=curr_score_name,
            hue=curr_score,
            palette=score_colors,
            linewidth=0.5,
            alpha=0.75,
            edgecolor="k",
            s=20,
            hue_order=list(score_colors.keys()),
            legend=True,
            ax=axs['21'],
        )
        axs['21'].set_xlim(x_min - 0.1 * x_ptp, x_max + 0.1 * x_ptp)
        axs['21'].set_ylim(y_min - 0.1 * y_ptp, y_max + 0.1 * y_ptp)
        scatter.legend_.set_title(curr_score_name)
        
        sns.violinplot(
            data=df_epi_scores,
            x=curr_score,
            y=f"{epiage} acceleration",
            hue=curr_score,
            palette=score_colors,
            density_norm='width',
            order=score_vals,
            saturation=0.75,
            linewidth=1.0,
            ax=axs['22'],
            legend=False,
            cut=0,
        )
        axs['22'].set_ylabel(f"{epiage} acceleration")
        axs['22'].set_xlabel(curr_score_name)
        mannwhitneyu_pval = df_stat.at[epiage, "mannwhitneyu_pval_fdr_bh"]
        levene_pval = df_stat.at[epiage, "levene_pval_fdr_bh"]
        title = f'Mann-Whitney: {mannwhitneyu_pval:.2e}\nLevene: {levene_pval:.2e}'
        for pval_col in pvals_cols_ancova:
            title += f"\nANCOVA: {df_stat.at[epiage, pval_col + '_fdr_bh']:.2e}"
        axs['22'].set_title(title)

    fig.savefig(f"{path}/05_scores/{curr_score_name}/continuous/dnam/all/ages_distribution_corrected.png", bbox_inches='tight', dpi=200)
    fig.savefig(f"{path}/05_scores/{curr_score_name}/continuous/dnam/all/ages_distribution_corrected.pdf", bbox_inches='tight')
    plt.close(fig)

C:\Users\alena\AppData\Local\Temp\ipykernel_38988\2490360474.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_epi_scores[f"{epiage}_linear_pred"] = linreg.predict(df_epi_scores)
C:\Users\alena\AppData\Local\Temp\ipykernel_38988\2490360474.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_epi_scores[f"{epiage} acceleration"] = df_epi_scores[epiage] - df_epi_scores[f"{epiage}_linear_pred"]
C:\Users\alena\AppData\Local\Temp\ipykernel_38988\2490360474.py:14: SettingWithCopyWarning: 
A value is tryi

In [10]:
# Here we take samples with low risk for linreg
for curr_score in pheno_associations:
    curr_score_name = curr_score.split(' group')[0]
    pathlib.Path(f"{path}/05_scores/{curr_score_name}/continuous/dnam/low_risk").mkdir(parents=True, exist_ok=True)
    df_epi_scores = df_epi.loc[df_epi[curr_score].isin(pheno_associations[curr_score]['groups'])]
    df_stat = pd.DataFrame(index=epi_ages)
    score_vals = pheno_associations[curr_score]['groups']
    score_val_base = pheno_associations[curr_score]['base']
    score_colors = pheno_associations[curr_score]['colors']

    for epiage_id, epiage in enumerate(epi_ages):
        
        linreg = smf.ols(formula=f"{epiage} ~ Age", data=df_epi_scores.loc[df_epi_scores[curr_score] == score_val_base, :]).fit()
        df_epi_scores[f"{epiage}_linear_pred"] = linreg.predict(df_epi_scores)
        df_epi_scores[f"{epiage} acceleration"] = df_epi_scores[epiage] - df_epi_scores[f"{epiage}_linear_pred"]
        vals = {}
        for group in score_vals:
            vals[group] = df_epi_scores.loc[df_epi_scores[curr_score] == group, f"{epiage} acceleration"].values
        _, df_stat.at[epiage, "mannwhitneyu_pval"] = mannwhitneyu(vals[score_vals[0]], vals[score_vals[1]], alternative='two-sided')
        _, df_stat.at[epiage, "levene_pval"] = levene(vals[score_vals[0]], vals[score_vals[1]])
        regcov = smf.ols(formula=f"{epiage} ~ Q('{curr_score}') + Age", data=df_epi_scores).fit()
        reg_sum = regcov.summary2().tables[1]
        pvals_cols = reg_sum.index[reg_sum.index.str.contains(curr_score_name)].values
        for pval_col_id, pval_col in enumerate(pvals_cols):
            df_stat.at[epiage, f"ancova_{pval_col}_pval"] = reg_sum.at[pval_col, 'P>|t|']
        
        _, df_stat.at[epiage, "pearson_pval"] = pearsonr(df_epi_scores.loc[df_epi_scores[curr_score] == score_val_base, :][curr_score_name].values, df_epi_scores.loc[df_epi_scores[curr_score] == score_val_base, :][f"{epiage} acceleration"].values)
        _, df_stat.at[epiage, "spearman_pval"] = spearmanr(df_epi_scores.loc[df_epi_scores[curr_score] == score_val_base, :][curr_score_name].values, df_epi_scores.loc[df_epi_scores[curr_score] == score_val_base, :][f"{epiage} acceleration"].values)

    _, df_stat.loc[epi_ages, "mannwhitneyu_pval_fdr_bh"], _, _ = multipletests(df_stat.loc[epi_ages, "mannwhitneyu_pval"].values, 0.05, method='fdr_bh')
    _, df_stat.loc[epi_ages, "levene_pval_fdr_bh"], _, _ = multipletests(df_stat.loc[epi_ages, "levene_pval"].values, 0.05, method='fdr_bh')
    pvals_cols_ancova = df_stat.columns[df_stat.columns.str.contains(curr_score_name)].values
    for pval_col in pvals_cols_ancova:
        _, df_stat.loc[epi_ages, f"{pval_col}_fdr_bh"], _, _ = multipletests(df_stat.loc[epi_ages, pval_col].values, 0.05, method='fdr_bh')
    
    _, df_stat.loc[epi_ages, "pearson_pval_fdr_bh"], _, _ = multipletests(df_stat.loc[epi_ages, "pearson_pval"].values, 0.05, method='fdr_bh')
    _, df_stat.loc[epi_ages, "spearman_pval_fdr_bh"], _, _ = multipletests(df_stat.loc[epi_ages, "spearman_pval"].values, 0.05, method='fdr_bh')
    
    hist_bins = np.linspace(5, 115, 23)
    fig, ax = plt.subplots(figsize=(6, 4))
    histplot = sns.histplot(
        data=df_epi_scores,
        bins=hist_bins,
        edgecolor='k',
        linewidth=1,
        x="Age",
        hue=curr_score,
        palette=score_colors,
        ax=ax
    )
    histplot.set(xlim=(0, 120))
    plt.savefig(f"{path}/05_scores/{curr_score_name}/continuous/dnam/low_risk/hist_age_corrected.png", bbox_inches='tight', dpi=200)
    plt.savefig(f"{path}/05_scores/{curr_score_name}/continuous/dnam/low_risk/hist_age_corrected.pdf", bbox_inches='tight')
    plt.close(fig)
        
    sns.set_theme(style='ticks')
    fig = plt.figure(
        figsize=(24, 28),
    )
    subfigs = fig.subfigures(
        nrows=7,
        ncols=4,
    )
    for epiage_id, epiage in enumerate(df_stat.index.values):
        row_id, col_id = divmod(epiage_id, 4)

        axs = subfigs[row_id, col_id].subplot_mosaic(
            [
                ['11', '12'],
                ['21', '22'],
            ],
            height_ratios=[1, 4],
            width_ratios=[3, 1.5],
            gridspec_kw={
                "bottom": 0.14,
                "top": 0.95,
                "wspace": 0.37,
                "hspace": 0.01,
            },
        )
        
        ds_table = pd.DataFrame(index=[fr"Pearson $\rho$", "Pearson p-value", fr"Spearman $\rho$", "Spearman p-value"], columns=[epiage])
        rho, _ = pearsonr(df_epi_scores.loc[df_epi_scores[curr_score] == score_val_base, :][curr_score_name].values, df_epi_scores.loc[df_epi_scores[curr_score] == score_val_base, :][f"{epiage} acceleration"].values)
        sp_rho, _ = spearmanr(df_epi_scores.loc[df_epi_scores[curr_score] == score_val_base, :][curr_score_name].values, df_epi_scores.loc[df_epi_scores[curr_score] == score_val_base, :][f"{epiage} acceleration"].values)
        pearson_pval = df_stat.at[epiage, "pearson_pval_fdr_bh"]
        spearman_pval = df_stat.at[epiage, "spearman_pval_fdr_bh"]
        ds_table.at[fr"Pearson $\rho$", epiage] = f"{rho:0.2f}"
        ds_table.at["Pearson p-value", epiage] = f"{pearson_pval:.2e}"
        ds_table.at[fr"Spearman $\rho$", epiage] = f"{sp_rho:0.2f}"
        ds_table.at["Spearman p-value", epiage] = f"{spearman_pval:.2e}"
        col_defs = [
            ColumnDefinition(
                name="index",
                title=epiage,
                textprops={"ha": "left"},
                width=4.5,
            ),
            ColumnDefinition(
                name=epiage,
                title='',
                textprops={"ha": "center"},
                width=2.0,
            ),
        ]
        table = Table(
            ds_table,
            column_definitions=col_defs,
            row_dividers=True,
            footer_divider=False,
            ax=axs['11'],
            textprops={"fontsize": 7},
            row_divider_kw={"linewidth": 1, "linestyle": (0, (1, 1))},
            col_label_divider_kw={"linewidth": 1, "linestyle": "-"},
            column_border_kw={"linewidth": 1, "linestyle": "-"},
        ).autoset_fontcolors(colnames=[epiage])
        
        axs['12'].axis('off')
        
        x_min = df_epi_scores[f"{epiage} acceleration"].min()
        x_max = df_epi_scores[f"{epiage} acceleration"].max()
        x_ptp = x_max - x_min
        y_min = df_epi_scores[curr_score_name].min()
        y_max = df_epi_scores[curr_score_name].max()
        y_ptp = y_max - y_min
        regplot = sns.regplot(
            data=df_epi_scores.loc[df_epi_scores[curr_score] == score_val_base, :],
            x=f'{epiage} acceleration',
            y=curr_score_name,
            color=score_colors[score_val_base],
            scatter=False,
            ax=axs['21']
        )
        scatter = sns.scatterplot(
            data=df_epi_scores,
            x=f"{epiage} acceleration",
            y=curr_score_name,
            hue=curr_score,
            palette=score_colors,
            linewidth=0.5,
            alpha=0.75,
            edgecolor="k",
            s=20,
            hue_order=list(score_colors.keys()),
            legend=True,
            ax=axs['21'],
        )
        axs['21'].set_xlim(x_min - 0.1 * x_ptp, x_max + 0.1 * x_ptp)
        axs['21'].set_ylim(y_min - 0.1 * y_ptp, y_max + 0.1 * y_ptp)
        scatter.legend_.set_title(curr_score_name)
        
        sns.violinplot(
            data=df_epi_scores,
            x=curr_score,
            y=f"{epiage} acceleration",
            hue=curr_score,
            palette=score_colors,
            density_norm='width',
            order=score_vals,
            saturation=0.75,
            linewidth=1.0,
            ax=axs['22'],
            legend=False,
            cut=0,
        )
        axs['22'].set_ylabel(f"{epiage} acceleration")
        axs['22'].set_xlabel(curr_score_name)
        mannwhitneyu_pval = df_stat.at[epiage, "mannwhitneyu_pval_fdr_bh"]
        levene_pval = df_stat.at[epiage, "levene_pval_fdr_bh"]
        title = f'Mann-Whitney: {mannwhitneyu_pval:.2e}\nLevene: {levene_pval:.2e}'
        for pval_col in pvals_cols_ancova:
            title += f"\nANCOVA: {df_stat.at[epiage, pval_col + '_fdr_bh']:.2e}"
        axs['22'].set_title(title)

    fig.savefig(f"{path}/05_scores/{curr_score_name}/continuous/dnam/low_risk/ages_distribution_corrected.png", bbox_inches='tight', dpi=200)
    fig.savefig(f"{path}/05_scores/{curr_score_name}/continuous/dnam/low_risk/ages_distribution_corrected.pdf", bbox_inches='tight')
    plt.close(fig)

C:\Users\alena\AppData\Local\Temp\ipykernel_38988\1012613408.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_epi_scores[f"{epiage}_linear_pred"] = linreg.predict(df_epi_scores)
C:\Users\alena\AppData\Local\Temp\ipykernel_38988\1012613408.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_epi_scores[f"{epiage} acceleration"] = df_epi_scores[epiage] - df_epi_scores[f"{epiage}_linear_pred"]
C:\Users\alena\AppData\Local\Temp\ipykernel_38988\1012613408.py:14: SettingWithCopyWarning: 
A value is tryi

## Epigenetic metrics

In [11]:
for curr_score in pheno_associations:
    curr_score_name = curr_score.split(' group')[0]
    pathlib.Path(f"{path}/05_scores/{curr_score_name}/categorical/dnam").mkdir(parents=True, exist_ok=True)
    df_epi_scores = df_epi.loc[df_epi[curr_score].isin(pheno_associations[curr_score]['groups'])]
    df_stat = pd.DataFrame(index=epi_metrics)
    score_vals = pheno_associations[curr_score]['groups']
    score_val_base = pheno_associations[curr_score]['base']
    score_colors = pheno_associations[curr_score]['colors']

    for epi_metric_id, epi_metric in enumerate(epi_metrics):
        pathlib.Path(f"{path}/05_scores/{curr_score_name}/categorical/dnam/").mkdir(parents=True, exist_ok=True)
        vals = {}
        for group in score_vals:
            vals[group] = df_epi_scores.loc[df_epi_scores[curr_score] == group, epi_metric].values
            df_stat.at[epi_metric, f"Mean {group}"] = np.mean(vals[group])
            df_stat.at[epi_metric, f"Median {group}"] = np.median(vals[group])
            df_stat.at[epi_metric, f"Q75 {group}"], df_stat.at[epi_metric, f"Q25 {group}"] = np.percentile(vals[group], [75 , 25])
            df_stat.at[epi_metric, f"IQR {group}"] = df_stat.at[epi_metric, f"Q75 {group}"] - df_stat.at[epi_metric, f"Q25 {group}"]
            df_stat.at[epi_metric, f"Variation {group}"] = variation(vals[group])
        _, df_stat.at[epi_metric, "mannwhitneyu_pval"] = mannwhitneyu(vals[score_vals[0]], vals[score_vals[1]], alternative='two-sided')
        _, df_stat.at[epi_metric, "levene_pval"] = levene(vals[score_vals[0]], vals[score_vals[1]])
        regcov = smf.ols(formula=f"{epi_metric} ~ Q('{curr_score}') + Age", data=df_epi_scores).fit()
        reg_sum = regcov.summary2().tables[1]
        pvals_cols = reg_sum.index[reg_sum.index.str.contains(curr_score_name)].values
        for pval_col_id, pval_col in enumerate(pvals_cols):
            df_stat.at[epi_metric, f"ancova_{pval_col}_pval"] = reg_sum.at[pval_col, 'P>|t|']
        
        _, df_stat.at[epi_metric, "pearson_pval"] = pearsonr(df_epi_scores[curr_score_name].values, df_epi_scores[epi_metric].values)
        _, df_stat.at[epi_metric, "spearman_pval"] = spearmanr(df_epi_scores[curr_score_name].values, df_epi_scores[epi_metric].values)

    _, df_stat.loc[epi_metrics, "mannwhitneyu_pval_fdr_bh"], _, _ = multipletests(df_stat.loc[epi_metrics, "mannwhitneyu_pval"].values, 0.05, method='fdr_bh')
    _, df_stat.loc[epi_metrics, "mannwhitneyu_pval_bonferroni"], _, _ = multipletests(df_stat.loc[epi_metrics, "mannwhitneyu_pval"].values, 0.05, method='bonferroni')
    _, df_stat.loc[epi_metrics, "mannwhitneyu_pval_simes-hochberg"], _, _ = multipletests(df_stat.loc[epi_metrics, "mannwhitneyu_pval"].values, 0.05, method='simes-hochberg')
    _, df_stat.loc[epi_metrics, "levene_pval_fdr_bh"], _, _ = multipletests(df_stat.loc[epi_metrics, "levene_pval"].values, 0.05, method='fdr_bh')
    _, df_stat.loc[epi_metrics, "levene_pval_bonferroni"], _, _ = multipletests(df_stat.loc[epi_metrics, "levene_pval"].values, 0.05, method='bonferroni')
    _, df_stat.loc[epi_metrics, "levene_pval_simes-hochberg"], _, _ = multipletests(df_stat.loc[epi_metrics, "levene_pval"].values, 0.05, method='simes-hochberg')
    pvals_cols_ancova = df_stat.columns[df_stat.columns.str.contains(curr_score_name)].values
    for pval_col in pvals_cols_ancova:
        _, df_stat.loc[epi_metrics, f"{pval_col}_fdr_bh"], _, _ = multipletests(df_stat.loc[epi_metrics, pval_col].values, 0.05, method='fdr_bh')
        _, df_stat.loc[epi_metrics, f"{pval_col}_bonferroni"], _, _ = multipletests(df_stat.loc[epi_metrics, pval_col].values, 0.05, method='bonferroni')
        _, df_stat.loc[epi_metrics, f"{pval_col}_simes-hochberg"], _, _ = multipletests(df_stat.loc[epi_metrics, pval_col].values, 0.05, method='simes-hochberg')
    
    _, df_stat.loc[epi_metrics, "pearson_pval_fdr_bh"], _, _ = multipletests(df_stat.loc[epi_metrics, "pearson_pval"].values, 0.05, method='fdr_bh')
    _, df_stat.loc[epi_metrics, "pearson_pval_bonferroni"], _, _ = multipletests(df_stat.loc[epi_metrics, "pearson_pval"].values, 0.05, method='bonferroni')
    _, df_stat.loc[epi_metrics, "pearson_pval_simes-hochberg"], _, _ = multipletests(df_stat.loc[epi_metrics, "pearson_pval"].values, 0.05, method='simes-hochberg')
    _, df_stat.loc[epi_metrics, "spearman_pval_fdr_bh"], _, _ = multipletests(df_stat.loc[epi_metrics, "spearman_pval"].values, 0.05, method='fdr_bh')
    _, df_stat.loc[epi_metrics, "spearman_pval_bonferroni"], _, _ = multipletests(df_stat.loc[epi_metrics, "spearman_pval"].values, 0.05, method='bonferroni')
    _, df_stat.loc[epi_metrics, "spearman_pval_simes-hochberg"], _, _ = multipletests(df_stat.loc[epi_metrics, "spearman_pval"].values, 0.05, method='simes-hochberg')

    df_stat.sort_values([f"mannwhitneyu_pval"], ascending=[True], inplace=True)
    df_stat.to_excel(f"{path}/05_scores/{curr_score_name}/categorical/dnam/metrics.xlsx")
    
    for stat_test in [x.replace('_pval', '') for x in pvals_cols_ancova] + ['mannwhitneyu', 'levene']:
        df_fig = df_stat.copy()
        df_fig.sort_values([f"{stat_test}_pval"], ascending=[True], inplace=True)
        df_fig['Features'] = df_fig.index
        df_fig[f'{stat_test}_pval_fdr_bh_log'] = -np.log10(df_fig[f'{stat_test}_pval_fdr_bh'])
        df_fig['color'] = 'pink'
        df_fig.loc[df_fig[f'{stat_test}_pval_fdr_bh'] < 0.05, 'color'] = 'red'
        sns.set_theme(style='ticks')
        fig, ax = plt.subplots(figsize=(3, df_fig.shape[0] * 0.5))
        barplot = sns.barplot(
            data=df_fig,
            y='Features',
            x=f'{stat_test}_pval_fdr_bh_log',
            edgecolor='black',
            palette=df_fig['color'].values,
            ax=ax,
        )
        ax.set_xlabel(r"$-\log_{10}(\mathrm{p-value})$")
        ax.xaxis.tick_top()
        ax.xaxis.set_label_position('top')
        ax.set_ylabel('')
        plt.savefig(f"{path}/05_scores/{curr_score_name}/categorical/dnam/metrics_pvals_{stat_test}.png", bbox_inches='tight', dpi=200)
        plt.savefig(f"{path}/05_scores/{curr_score_name}/categorical/dnam/metrics_pvals_{stat_test}.pdf", bbox_inches='tight')
        plt.close(fig)
        
    sns.set_theme(style='ticks')
    fig = plt.figure(
        figsize=(18, 8),
    )
    subfigs = fig.subfigures(
        nrows=2,
        ncols=3,
    )
    for epi_metric_id, epi_metric in enumerate(df_stat.index.values):
        row_id, col_id = divmod(epi_metric_id, 3)

        axs = subfigs[row_id, col_id].subplot_mosaic(
            [
                ['11', '12'],
                ['21', '22'],
            ],
            height_ratios=[1, 4],
            width_ratios=[3, 1.5],
            gridspec_kw={
                "bottom": 0.14,
                "top": 0.95,
                "wspace": 0.45,
                "hspace": 0.01,
            },
        )
        
        ds_table = pd.DataFrame(index=[fr"Pearson $\rho$", "Pearson p-value", fr"Spearman $\rho$", "Spearman p-value"], columns=[epi_metric])
        rho, _ = pearsonr(df_epi_scores[curr_score_name].values, df_epi_scores[epi_metric].values)
        sp_rho, _ = spearmanr(df_epi_scores[curr_score_name].values, df_epi_scores[epi_metric].values)
        pearson_pval = df_stat.at[epi_metric, "pearson_pval_fdr_bh"]
        spearman_pval = df_stat.at[epi_metric, "spearman_pval_fdr_bh"]
        ds_table.at[fr"Pearson $\rho$", epi_metric] = f"{rho:0.2f}"
        ds_table.at["Pearson p-value", epi_metric] = f"{pearson_pval:.2e}"
        ds_table.at[fr"Spearman $\rho$", epi_metric] = f"{sp_rho:0.2f}"
        ds_table.at["Spearman p-value", epi_metric] = f"{spearman_pval:.2e}"
        col_defs = [
            ColumnDefinition(
                name="index",
                title=epi_metric,
                textprops={"ha": "left"},
                width=4.5,
            ),
            ColumnDefinition(
                name=epi_metric,
                title='',
                textprops={"ha": "center"},
                width=2.0,
            ),
        ]
        table = Table(
            ds_table,
            column_definitions=col_defs,
            row_dividers=True,
            footer_divider=False,
            ax=axs['11'],
            textprops={"fontsize": 7},
            row_divider_kw={"linewidth": 1, "linestyle": (0, (1, 1))},
            col_label_divider_kw={"linewidth": 1, "linestyle": "-"},
            column_border_kw={"linewidth": 1, "linestyle": "-"},
        ).autoset_fontcolors(colnames=[epi_metric])
        
        axs['12'].axis('off')
        
        x_min = df_epi_scores[epi_metric].min()
        x_max = df_epi_scores[epi_metric].max()
        x_ptp = x_max - x_min
        y_min = df_epi_scores[curr_score_name].min()
        y_max = df_epi_scores[curr_score_name].max()
        y_ptp = y_max - y_min
        regplot = sns.regplot(
            data=df_epi_scores,
            x=epi_metric,
            y=curr_score_name,
            color='dimgray',
            scatter=False,
            ax=axs['21']
        )
        scatter = sns.scatterplot(
            data=df_epi_scores,
            x=epi_metric,
            y=curr_score_name,
            hue=curr_score,
            palette=score_colors,
            linewidth=0.5,
            alpha=0.75,
            edgecolor="k",
            s=20,
            hue_order=list(score_colors.keys()),
            legend=True,
            ax=axs['21'],
        )
        axs['21'].set_xlim(x_min - 0.1 * x_ptp, x_max + 0.1 * x_ptp)
        axs['21'].set_ylim(y_min - 0.1 * y_ptp, y_max + 0.1 * y_ptp)
        scatter.legend_.set_title(curr_score_name)
        
        sns.violinplot(
            data=df_epi_scores,
            x=curr_score,
            y=epi_metric,
            hue=curr_score,
            palette=score_colors,
            density_norm='width',
            order=score_vals,
            saturation=0.75,
            linewidth=1.0,
            ax=axs['22'],
            legend=False,
            cut=0,
        )
        axs['22'].set_ylabel(epi_metric)
        axs['22'].set_xlabel(curr_score_name)
        mannwhitneyu_pval = df_stat.at[epi_metric, "mannwhitneyu_pval_fdr_bh"]
        levene_pval = df_stat.at[epi_metric, "levene_pval_fdr_bh"]
        title = f'Mann-Whitney: {mannwhitneyu_pval:.2e}\nLevene: {levene_pval:.2e}'
        for pval_col in pvals_cols_ancova:
            title += f"\nANCOVA: {df_stat.at[epi_metric, pval_col + '_fdr_bh']:.2e}"
        axs['22'].set_title(title)

    fig.savefig(f"{path}/05_scores/{curr_score_name}/categorical/dnam/metrics_distribution_corrected.png", bbox_inches='tight', dpi=200)
    fig.savefig(f"{path}/05_scores/{curr_score_name}/categorical/dnam/metrics_distribution_corrected.pdf", bbox_inches='tight')
    plt.close(fig)

C:\Users\alena\AppData\Local\Temp\ipykernel_38988\2266329000.py:62: FutureWarning: 

Passing `palette` without assigning `hue` is deprecated and will be removed in v0.14.0. Assign the `y` variable to `hue` and set `legend=False` for the same effect.

  barplot = sns.barplot(
C:\Users\alena\AppData\Local\Temp\ipykernel_38988\2266329000.py:62: UserWarning: Numpy array is not a supported type for `palette`. Please convert your palette to a list. This will become an error in v0.14
  barplot = sns.barplot(
C:\Users\alena\AppData\Local\Temp\ipykernel_38988\2266329000.py:62: FutureWarning: 

Passing `palette` without assigning `hue` is deprecated and will be removed in v0.14.0. Assign the `y` variable to `hue` and set `legend=False` for the same effect.

  barplot = sns.barplot(
C:\Users\alena\AppData\Local\Temp\ipykernel_38988\2266329000.py:62: UserWarning: Numpy array is not a supported type for `palette`. Please convert your palette to a list. This will become an error in v0.14
  barplot =

# PhenoAge & CognitiveAge

## Load data

In [12]:
path = f"E:/YandexDisk/bbd/fmba"

df_pheno = pd.read_excel(f"{path}/pheno_fixed.xlsx", index_col=0)
df_pheno.index = df_pheno.index.astype(str)
cols_pheno = ['дата рождения']

df_ages = pd.read_excel(f"{path}/PhenoAge_CognitiveAge/df.xlsx", index_col=0)
df_ages.index = df_ages.index.astype(str)
cols_ages = [
    'Chronological Age (PhenoAge)',
    'PhenoAge',
    'Chronological Age (CognitiveAge)',
    'CognitiveAge'
]

df_phenoage_nmk = pd.read_excel(f"{path}/03_pheno_age/data_PhenoAge.xlsx", index_col=0)
df_phenoage_nmk.index = df_phenoage_nmk.index.astype(str)
df_phenoage_nmk.rename(columns={
    'PhenoAge': 'PhenoAge nmk',
    }, inplace=True
)
cols_phenoage_nmk = [
    'PhenoAge nmk',
]

df_scores = pd.read_excel(f"{path}/05_scores/df_scores.xlsx", index_col=0)
df_scores.index = df_scores.index.astype(str)
cols_scores = [#'SCORE', 'SCORE group thr=0.02', 'SCORE2 wo HDL', 'SCORE2 wo HDL group thr=0.05', 'SCORE2 aver HDL', 'SCORE2 aver HDL group thr=0.05', 
               #'FHR wo HDL', 'FHR wo HDL group thr=10.0', 'FHR wo HDL Heart Age Acceleration', 'FHR wo HDL Heart Age Acceleration group thr=0.0',
               #'FHR aver HDL', 'FHR aver HDL group thr=10.0', 'FHR aver HDL Heart Age Acceleration', 'FHR aver HDL Heart Age Acceleration group thr=0.0',
               #'ASCVD wo HDL', 'ASCVD wo HDL group thr=1.0', 'ASCVD aver HDL', 'ASCVD aver HDL group thr=2.0',
               #'Reynolds wo HDL', 'Reynolds wo HDL group thr=2.0', 'Reynolds aver HDL', 'Reynolds aver HDL group thr=2.0',
               'CKD-EPI', 'CKD-EPI group thr=75.0']

dfs = [df_pheno[cols_pheno], df_ages[cols_ages], df_phenoage_nmk[cols_phenoage_nmk], df_scores[cols_scores]]
df = pd.concat(dfs, axis=1)
df['дата рождения'] = pd.to_datetime(df['дата рождения'])
df['Age'] = (pd.to_datetime("2024-11-11") - df['дата рождения']) / np.timedelta64(1, 'D') / 365.25
df['Chronological Age (PhenoAge nmk)'] = df['Age']
df.drop(['3278', '4436'])

pheno_associations = {
    #'SCORE group thr=0.02': {
    #    'groups': ['Low', 'High'],
    #    'base': 'Low',
    #    'colors': {'Low': 'dodgerblue', 'High': 'crimson'}
    #},
    #'SCORE2 wo HDL group thr=0.05': {
    #    'groups': ['Low', 'High'],
    #    'base': 'Low',
    #    'colors': {'Low': 'dodgerblue', 'High': 'crimson'}
    #},
    #'SCORE2 aver HDL group thr=0.05': {
    #    'groups': ['Low', 'High'],
    #    'base': 'Low',
    #    'colors': {'Low': 'dodgerblue', 'High': 'crimson'}
    #},
    #'FHR wo HDL group thr=10.0': {
    #    'groups': ['Low', 'High'],
    #    'base': 'Low',
    #    'colors': {'Low': 'dodgerblue', 'High': 'crimson'}
    #},
    #'FHR wo HDL Heart Age Acceleration group thr=0.0': {
    #    'groups': ['Low', 'High'],
    #    'base': 'Low',
    #    'colors': {'Low': 'dodgerblue', 'High': 'crimson'}
    #},
    #'FHR aver HDL group thr=10.0': {
    #    'groups': ['Low', 'High'],
    #    'base': 'Low',
    #    'colors': {'Low': 'dodgerblue', 'High': 'crimson'}
    #},
    #'FHR aver HDL Heart Age Acceleration group thr=0.0': {
    #    'groups': ['Low', 'High'],
    #    'base': 'Low',
    #    'colors': {'Low': 'dodgerblue', 'High': 'crimson'}
    #},
    #'ASCVD wo HDL group thr=1.0': {
    #    'groups': ['Low', 'High'],
    #    'base': 'Low',
    #    'colors': {'Low': 'dodgerblue', 'High': 'crimson'}
    #},
    #'ASCVD aver HDL group thr=2.0': {
    #    'groups': ['Low', 'High'],
    #    'base': 'Low',
    #    'colors': {'Low': 'dodgerblue', 'High': 'crimson'}
    #},
    #'Reynolds wo HDL group thr=2.0': {
    #    'groups': ['Low', 'High'],
    #    'base': 'Low',
    #    'colors': {'Low': 'dodgerblue', 'High': 'crimson'}
    #},
    #'Reynolds aver HDL group thr=2.0': {
    #    'groups': ['Low', 'High'],
    #    'base': 'Low',
    #    'colors': {'Low': 'dodgerblue', 'High': 'crimson'}
    #},
    'CKD-EPI group thr=75.0': {
        'groups': ['Low', 'High'],
        'base': 'Low',
        'colors': {'Low': 'dodgerblue', 'High': 'crimson'}
    },
}

## Ages (categorical score)

### Casual acceleration

In [13]:
# Here we take all samples for linreg
ages = ['PhenoAge', 'PhenoAge nmk', 'CognitiveAge']

for curr_score in pheno_associations:
    curr_score_name = curr_score.split(' group')[0]
    df_stat = pd.DataFrame(index=ages)
    score_vals = pheno_associations[curr_score]['groups']
    score_val_base = pheno_associations[curr_score]['base']
    score_colors = pheno_associations[curr_score]['colors']
    
    for age in ages:

        pathlib.Path(f"{path}/05_scores/{curr_score_name}/categorical/{age}/casual_acc/all").mkdir(parents=True, exist_ok=True)
        
        df_age_scores = df.dropna(subset=[age, curr_score])

        linreg = smf.ols(formula=f"Q('{age}') ~ Q('Chronological Age ({age})')", data=df_age_scores).fit()
        df_age_scores[f"{age} linear pred"] = linreg.predict(df_age_scores)
        df_age_scores[f"{age} acceleration"] = df_age_scores[age] - df_age_scores[f"{age} linear pred"]
        vals = {}
        for group in score_vals:
            vals[group] = df_age_scores.loc[df_age_scores[curr_score] == group, f"{age} acceleration"].values
        _, df_stat.at[age, "mannwhitneyu_pval"] = mannwhitneyu(vals[score_vals[0]], vals[score_vals[1]], alternative='two-sided')
        _, df_stat.at[age, "levene_pval"] = levene(vals[score_vals[0]], vals[score_vals[1]])
        regcov = smf.ols(formula=f"Q('{age}') ~ Q('{curr_score}') + Age", data=df_age_scores).fit()
        reg_sum = regcov.summary2().tables[1]
        pvals_cols = reg_sum.index[reg_sum.index.str.contains(curr_score_name)].values
        for pval_col_id, pval_col in enumerate(pvals_cols):
            df_stat.at[age, f"ancova_{pval_col}_pval"] = reg_sum.at[pval_col, 'P>|t|']

        hist_bins = np.linspace(5, 115, 23)
        fig, ax = plt.subplots(figsize=(6, 4))
        histplot = sns.histplot(
            data=df_age_scores,
            bins=hist_bins,
            edgecolor='k',
            linewidth=1,
            x="Age",
            hue=curr_score,
            palette=score_colors,
            ax=ax
        )
        histplot.set(xlim=(0, 120))
        plt.savefig(f"{path}/05_scores/{curr_score_name}/categorical/{age}/casual_acc/all/hist_age_corrected.png", bbox_inches='tight', dpi=200)
        plt.savefig(f"{path}/05_scores/{curr_score_name}/categorical/{age}/casual_acc/all/hist_age_corrected.pdf", bbox_inches='tight')
        plt.close(fig)

        fig, axs = plt.subplot_mosaic(
            [
                ['11', '12'],
                ['21', '22'],
            ],
            figsize=(8, 6),
            height_ratios=[1, 4],
            width_ratios=[3, 1.5],
            gridspec_kw={
                "bottom": 0.14,
                "top": 0.95,
                "wspace": 0.33,
                "hspace": 0.01,
            },
        )
        
        ds_table = pd.DataFrame(index=[fr"Pearson $\rho$", "Pearson p-value", fr"Spearman $\rho$", "Spearman p-value"], columns=[age])
        rho, pval = pearsonr(df_age_scores['Age'].values, df_age_scores[age].values)
        sp_rho, sp_pval = spearmanr(df_age_scores['Age'].values, df_age_scores[age].values)
        ds_table.at[fr"Pearson $\rho$", age] = f"{rho:0.2f}"
        ds_table.at["Pearson p-value", age] = f"{pval:.2e}"
        ds_table.at[fr"Spearman $\rho$", age] = f"{sp_rho:0.2f}"
        ds_table.at["Spearman p-value", age] = f"{sp_pval:.2e}"
        col_defs = [
            ColumnDefinition(
                name="index",
                title=age,
                textprops={"ha": "left"},
                width=4.5,
            ),
            ColumnDefinition(
                name=age,
                title='',
                textprops={"ha": "center"},
                width=2.0,
            ),
        ]
        table = Table(
            ds_table,
            column_definitions=col_defs,
            row_dividers=True,
            footer_divider=False,
            ax=axs['11'],
            textprops={"fontsize": 7},
            row_divider_kw={"linewidth": 1, "linestyle": (0, (1, 1))},
            col_label_divider_kw={"linewidth": 1, "linestyle": "-"},
            column_border_kw={"linewidth": 1, "linestyle": "-"},
        ).autoset_fontcolors(colnames=[age])
        
        axs['12'].axis('off')
        
        xy_min = df_age_scores[[f'Chronological Age ({age})', age]].min().min()
        xy_max = df_age_scores[[f'Chronological Age ({age})', age]].max().max()
        xy_ptp = xy_max - xy_min
        bisect = sns.lineplot(
            x=[xy_min - 0.1 * xy_ptp, xy_max + 0.1 * xy_ptp],
            y=[xy_min - 0.1 * xy_ptp, xy_max + 0.1 * xy_ptp],
            linestyle='--',
            color='black',
            linewidth=1.0,
            ax=axs['21']
        )
        regplot = sns.regplot(
            data=df_age_scores,
            x=f'Chronological Age ({age})',
            y=age,
            color='dimgray',
            scatter=False,
            truncate=False,
            ax=axs['21']
        )
        scatter = sns.scatterplot(
            data=df_age_scores,
            x=f'Chronological Age ({age})',
            y=age,
            hue=curr_score,
            palette=score_colors,
            linewidth=0.5,
            alpha=0.75,
            edgecolor="k",
            s=20,
            hue_order=list(score_colors.keys()),
            legend=True,
            ax=axs['21'],
        )
        axs['21'].set_xlabel(f"Age")
        axs['21'].set_xlim(xy_min - 0.1 * xy_ptp, xy_max + 0.1 * xy_ptp)
        axs['21'].set_ylim(xy_min - 0.1 * xy_ptp, xy_max + 0.1 * xy_ptp)
        scatter.legend_.set_title(curr_score_name)
        
        sns.violinplot(
            data=df_age_scores,
            x=curr_score,
            y=f"{age} acceleration",
            hue=curr_score,
            palette=score_colors,
            density_norm='width',
            order=score_vals,
            saturation=0.75,
            linewidth=1.0,
            ax=axs['22'],
            legend=False,
            cut=0,
        )
        axs['22'].set_ylabel(f"{age} acceleration")
        axs['22'].set_xlabel(curr_score_name)
        mannwhitneyu_pval = df_stat.at[age, "mannwhitneyu_pval"]
        levene_pval = df_stat.at[age, "levene_pval"]
        title = f'Mann-Whitney: {mannwhitneyu_pval:.2e}\nLevene: {levene_pval:.2e}'
        for pval_col in pvals_cols:
            title += f"\nANCOVA: {df_stat.at[age, f'ancova_{pval_col}_pval']:.2e}"
        axs['22'].set_title(title)

        fig.savefig(f"{path}/05_scores/{curr_score_name}/categorical/{age}/casual_acc/all/{age}.png", bbox_inches='tight', dpi=200)
        fig.savefig(f"{path}/05_scores/{curr_score_name}/categorical/{age}/casual_acc/all/{age}.pdf", bbox_inches='tight')
        plt.close(fig)

C:\Users\alena\AppData\Local\Temp\ipykernel_38988\285599002.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_age_scores[f"{age} linear pred"] = linreg.predict(df_age_scores)
C:\Users\alena\AppData\Local\Temp\ipykernel_38988\285599002.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_age_scores[f"{age} acceleration"] = df_age_scores[age] - df_age_scores[f"{age} linear pred"]
C:\Users\alena\AppData\Local\Temp\ipykernel_38988\285599002.py:18: SettingWithCopyWarning: 
A value is trying to be set on

In [14]:
# Here we take samples with low risk for linreg
ages = ['PhenoAge', 'PhenoAge nmk', 'CognitiveAge']

for curr_score in pheno_associations:
    curr_score_name = curr_score.split(' group')[0]
    df_stat = pd.DataFrame(index=ages)
    score_vals = pheno_associations[curr_score]['groups']
    score_val_base = pheno_associations[curr_score]['base']
    score_colors = pheno_associations[curr_score]['colors']
    
    for age in ages:

        pathlib.Path(f"{path}/05_scores/{curr_score_name}/categorical/{age}/casual_acc/low_risk").mkdir(parents=True, exist_ok=True)
        
        df_age_scores = df.dropna(subset=[age, curr_score])

        linreg = smf.ols(formula=f"Q('{age}') ~ Q('Chronological Age ({age})')", data=df_age_scores.loc[df_age_scores[curr_score] == score_val_base, :]).fit()
        df_age_scores[f"{age} linear pred"] = linreg.predict(df_age_scores)
        df_age_scores[f"{age} acceleration"] = df_age_scores[age] - df_age_scores[f"{age} linear pred"]
        vals = {}
        for group in score_vals:
            vals[group] = df_age_scores.loc[df_age_scores[curr_score] == group, f"{age} acceleration"].values
        _, df_stat.at[age, "mannwhitneyu_pval"] = mannwhitneyu(vals[score_vals[0]], vals[score_vals[1]], alternative='two-sided')
        _, df_stat.at[age, "levene_pval"] = levene(vals[score_vals[0]], vals[score_vals[1]])
        regcov = smf.ols(formula=f"Q('{age}') ~ Q('{curr_score}') + Age", data=df_age_scores).fit()
        reg_sum = regcov.summary2().tables[1]
        pvals_cols = reg_sum.index[reg_sum.index.str.contains(curr_score_name)].values
        for pval_col_id, pval_col in enumerate(pvals_cols):
            df_stat.at[age, f"ancova_{pval_col}_pval"] = reg_sum.at[pval_col, 'P>|t|']

        hist_bins = np.linspace(5, 115, 23)
        fig, ax = plt.subplots(figsize=(6, 4))
        histplot = sns.histplot(
            data=df_age_scores,
            bins=hist_bins,
            edgecolor='k',
            linewidth=1,
            x="Age",
            hue=curr_score,
            palette=score_colors,
            ax=ax
        )
        histplot.set(xlim=(0, 120))
        plt.savefig(f"{path}/05_scores/{curr_score_name}/categorical/{age}/casual_acc/low_risk/hist_age_corrected.png", bbox_inches='tight', dpi=200)
        plt.savefig(f"{path}/05_scores/{curr_score_name}/categorical/{age}/casual_acc/low_risk/hist_age_corrected.pdf", bbox_inches='tight')
        plt.close(fig)

        fig, axs = plt.subplot_mosaic(
            [
                ['11', '12'],
                ['21', '22'],
            ],
            figsize=(8, 6),
            height_ratios=[1, 4],
            width_ratios=[3, 1.5],
            gridspec_kw={
                "bottom": 0.14,
                "top": 0.95,
                "wspace": 0.33,
                "hspace": 0.01,
            },
        )
        
        ds_table = pd.DataFrame(index=[fr"Pearson $\rho$", "Pearson p-value", fr"Spearman $\rho$", "Spearman p-value"], columns=[age])
        rho, pval = pearsonr(df_age_scores.loc[df_age_scores[curr_score] == score_val_base, :][f'Chronological Age ({age})'].values, df_age_scores.loc[df_age_scores[curr_score] == score_val_base, :][age].values)
        sp_rho, sp_pval = spearmanr(df_age_scores.loc[df_age_scores[curr_score] == score_val_base, :][f'Chronological Age ({age})'].values, df_age_scores.loc[df_age_scores[curr_score] == score_val_base, :][age].values)
        ds_table.at[fr"Pearson $\rho$", age] = f"{rho:0.2f}"
        ds_table.at["Pearson p-value", age] = f"{pval:.2e}"
        ds_table.at[fr"Spearman $\rho$", age] = f"{sp_rho:0.2f}"
        ds_table.at["Spearman p-value", age] = f"{sp_pval:.2e}"
        col_defs = [
            ColumnDefinition(
                name="index",
                title=age,
                textprops={"ha": "left"},
                width=4.5,
            ),
            ColumnDefinition(
                name=age,
                title='',
                textprops={"ha": "center"},
                width=2.0,
            ),
        ]
        table = Table(
            ds_table,
            column_definitions=col_defs,
            row_dividers=True,
            footer_divider=False,
            ax=axs['11'],
            textprops={"fontsize": 7},
            row_divider_kw={"linewidth": 1, "linestyle": (0, (1, 1))},
            col_label_divider_kw={"linewidth": 1, "linestyle": "-"},
            column_border_kw={"linewidth": 1, "linestyle": "-"},
        ).autoset_fontcolors(colnames=[age])
        
        axs['12'].axis('off')
        
        xy_min = df_age_scores[[f'Chronological Age ({age})', age]].min().min()
        xy_max = df_age_scores[[f'Chronological Age ({age})', age]].max().max()
        xy_ptp = xy_max - xy_min
        bisect = sns.lineplot(
            x=[xy_min - 0.1 * xy_ptp, xy_max + 0.1 * xy_ptp],
            y=[xy_min - 0.1 * xy_ptp, xy_max + 0.1 * xy_ptp],
            linestyle='--',
            color='black',
            linewidth=1.0,
            ax=axs['21']
        )
        regplot = sns.regplot(
            data=df_age_scores.loc[df_age_scores[curr_score] == score_val_base, :],
            x=f'Chronological Age ({age})',
            y=age,
            color=score_colors[score_val_base],
            scatter=False,
            truncate=False,
            ax=axs['21']
        )
        scatter = sns.scatterplot(
            data=df_age_scores,
            x=f'Chronological Age ({age})',
            y=age,
            hue=curr_score,
            palette=score_colors,
            linewidth=0.5,
            alpha=0.75,
            edgecolor="k",
            s=20,
            hue_order=list(score_colors.keys()),
            legend=True,
            ax=axs['21'],
        )
        axs['21'].set_xlabel(f"Age")
        axs['21'].set_xlim(xy_min - 0.1 * xy_ptp, xy_max + 0.1 * xy_ptp)
        axs['21'].set_ylim(xy_min - 0.1 * xy_ptp, xy_max + 0.1 * xy_ptp)
        scatter.legend_.set_title(curr_score_name)
        
        sns.violinplot(
            data=df_age_scores,
            x=curr_score,
            y=f"{age} acceleration",
            hue=curr_score,
            palette=score_colors,
            density_norm='width',
            order=score_vals,
            saturation=0.75,
            linewidth=1.0,
            ax=axs['22'],
            legend=False,
            cut=0,
        )
        axs['22'].set_ylabel(f"{age} acceleration")
        axs['22'].set_xlabel(curr_score_name)
        mannwhitneyu_pval = df_stat.at[age, "mannwhitneyu_pval"]
        levene_pval = df_stat.at[age, "levene_pval"]
        title = f'Mann-Whitney: {mannwhitneyu_pval:.2e}\nLevene: {levene_pval:.2e}'
        for pval_col in pvals_cols:
            title += f"\nANCOVA: {df_stat.at[age, f'ancova_{pval_col}_pval']:.2e}"
        axs['22'].set_title(title)

        fig.savefig(f"{path}/05_scores/{curr_score_name}/categorical/{age}/casual_acc/low_risk/{age}.png", bbox_inches='tight', dpi=200)
        fig.savefig(f"{path}/05_scores/{curr_score_name}/categorical/{age}/casual_acc/low_risk/{age}.pdf", bbox_inches='tight')
        plt.close(fig)

C:\Users\alena\AppData\Local\Temp\ipykernel_38988\3442088977.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_age_scores[f"{age} linear pred"] = linreg.predict(df_age_scores)
C:\Users\alena\AppData\Local\Temp\ipykernel_38988\3442088977.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_age_scores[f"{age} acceleration"] = df_age_scores[age] - df_age_scores[f"{age} linear pred"]
C:\Users\alena\AppData\Local\Temp\ipykernel_38988\3442088977.py:18: SettingWithCopyWarning: 
A value is trying to be set

### Epi-like acceleration

In [15]:
# Here we take all samples for linreg
ages = ['PhenoAge', 'PhenoAge nmk', 'CognitiveAge']

for curr_score in pheno_associations:
    curr_score_name = curr_score.split(' group')[0]
    df_stat = pd.DataFrame(index=ages)
    score_vals = pheno_associations[curr_score]['groups']
    score_val_base = pheno_associations[curr_score]['base']
    score_colors = pheno_associations[curr_score]['colors']
    
    for age in ages:

        pathlib.Path(f"{path}/05_scores/{curr_score_name}/categorical/{age}/epi_like_acc/all").mkdir(parents=True, exist_ok=True)
        
        df_age_scores = df.dropna(subset=[age, curr_score])

        linreg_cx = smf.ols(formula=f"Q('{age}') ~ Q('Chronological Age ({age})')", data=df_age_scores).fit()
        df_age_scores[f"{age}_linear_pred_cx"] = linreg_cx.predict(df_age_scores)
        df_age_scores[f"{age}_acceleration_cx"] = df_age_scores[age] - df_age_scores[f"{age}_linear_pred_cx"]
        df_age_scores[f"{age}"] = df_age_scores[f'Chronological Age ({age})'] + df_age_scores[f"{age}_acceleration_cx"]

        linreg = smf.ols(formula=f"Q('{age}') ~ Q('Chronological Age ({age})')", data=df_age_scores).fit()
        df_age_scores[f"{age} linear pred"] = linreg.predict(df_age_scores)
        df_age_scores[f"{age} acceleration"] = df_age_scores[age] - df_age_scores[f"{age} linear pred"]
        vals = {}
        for group in score_vals:
            vals[group] = df_age_scores.loc[df_age_scores[curr_score] == group, f"{age} acceleration"].values
        _, df_stat.at[age, "mannwhitneyu_pval"] = mannwhitneyu(vals[score_vals[0]], vals[score_vals[1]], alternative='two-sided')
        _, df_stat.at[age, "levene_pval"] = levene(vals[score_vals[0]], vals[score_vals[1]])
        regcov = smf.ols(formula=f"Q('{age}') ~ Q('{curr_score}') + Age", data=df_age_scores).fit()
        reg_sum = regcov.summary2().tables[1]
        pvals_cols = reg_sum.index[reg_sum.index.str.contains(curr_score_name)].values
        for pval_col_id, pval_col in enumerate(pvals_cols):
            df_stat.at[age, f"ancova_{pval_col}_pval"] = reg_sum.at[pval_col, 'P>|t|']

        hist_bins = np.linspace(5, 115, 23)
        fig, ax = plt.subplots(figsize=(6, 4))
        histplot = sns.histplot(
            data=df_age_scores,
            bins=hist_bins,
            edgecolor='k',
            linewidth=1,
            x="Age",
            hue=curr_score,
            palette=score_colors,
            ax=ax
        )
        histplot.set(xlim=(0, 120))
        plt.savefig(f"{path}/05_scores/{curr_score_name}/categorical/{age}/epi_like_acc/all/hist_age_corrected.png", bbox_inches='tight', dpi=200)
        plt.savefig(f"{path}/05_scores/{curr_score_name}/categorical/{age}/epi_like_acc/all/hist_age_corrected.pdf", bbox_inches='tight')
        plt.close(fig)

        fig, axs = plt.subplot_mosaic(
            [
                ['11', '12'],
                ['21', '22'],
            ],
            figsize=(8, 6),
            height_ratios=[1, 4],
            width_ratios=[3, 1.5],
            gridspec_kw={
                "bottom": 0.14,
                "top": 0.95,
                "wspace": 0.33,
                "hspace": 0.01,
            },
        )
        
        ds_table = pd.DataFrame(index=[fr"Pearson $\rho$", "Pearson p-value", fr"Spearman $\rho$", "Spearman p-value"], columns=[age])
        rho, pval = pearsonr(df_age_scores[f'Chronological Age ({age})'].values, df_age_scores[age].values)
        sp_rho, sp_pval = spearmanr(df_age_scores[f'Chronological Age ({age})'].values, df_age_scores[age].values)
        ds_table.at[fr"Pearson $\rho$", age] = f"{rho:0.2f}"
        ds_table.at["Pearson p-value", age] = f"{pval:.2e}"
        ds_table.at[fr"Spearman $\rho$", age] = f"{sp_rho:0.2f}"
        ds_table.at["Spearman p-value", age] = f"{sp_pval:.2e}"
        col_defs = [
            ColumnDefinition(
                name="index",
                title=age,
                textprops={"ha": "left"},
                width=4.5,
            ),
            ColumnDefinition(
                name=age,
                title='',
                textprops={"ha": "center"},
                width=2.0,
            ),
        ]
        table = Table(
            ds_table,
            column_definitions=col_defs,
            row_dividers=True,
            footer_divider=False,
            ax=axs['11'],
            textprops={"fontsize": 7},
            row_divider_kw={"linewidth": 1, "linestyle": (0, (1, 1))},
            col_label_divider_kw={"linewidth": 1, "linestyle": "-"},
            column_border_kw={"linewidth": 1, "linestyle": "-"},
        ).autoset_fontcolors(colnames=[age])
        
        axs['12'].axis('off')
        
        xy_min = df_age_scores[[f'Chronological Age ({age})', age]].min().min()
        xy_max = df_age_scores[[f'Chronological Age ({age})', age]].max().max()
        xy_ptp = xy_max - xy_min
        bisect = sns.lineplot(
            x=[xy_min - 0.1 * xy_ptp, xy_max + 0.1 * xy_ptp],
            y=[xy_min - 0.1 * xy_ptp, xy_max + 0.1 * xy_ptp],
            linestyle='--',
            color='black',
            linewidth=1.0,
            ax=axs['21']
        )
        regplot = sns.regplot(
            data=df_age_scores,
            x=f'Chronological Age ({age})',
            y=age,
            color='dimgray',
            scatter=False,
            truncate=False,
            ax=axs['21']
        )
        scatter = sns.scatterplot(
            data=df_age_scores,
            x=f'Chronological Age ({age})',
            y=age,
            hue=curr_score,
            palette=score_colors,
            linewidth=0.5,
            alpha=0.75,
            edgecolor="k",
            s=20,
            hue_order=list(score_colors.keys()),
            legend=True,
            ax=axs['21'],
        )
        axs['21'].set_xlabel(f"Age")
        axs['21'].set_xlim(xy_min - 0.1 * xy_ptp, xy_max + 0.1 * xy_ptp)
        axs['21'].set_ylim(xy_min - 0.1 * xy_ptp, xy_max + 0.1 * xy_ptp)
        scatter.legend_.set_title(curr_score_name)
        
        sns.violinplot(
            data=df_age_scores,
            x=curr_score,
            y=f"{age} acceleration",
            hue=curr_score,
            palette=score_colors,
            density_norm='width',
            order=score_vals,
            saturation=0.75,
            linewidth=1.0,
            ax=axs['22'],
            legend=False,
            cut=0,
        )
        axs['22'].set_ylabel(f"{age} acceleration")
        axs['22'].set_xlabel(curr_score_name)
        mannwhitneyu_pval = df_stat.at[age, "mannwhitneyu_pval"]
        levene_pval = df_stat.at[age, "levene_pval"]
        title = f'Mann-Whitney: {mannwhitneyu_pval:.2e}\nLevene: {levene_pval:.2e}'
        for pval_col in pvals_cols:
            title += f"\nANCOVA: {df_stat.at[age, f'ancova_{pval_col}_pval']:.2e}"
        axs['22'].set_title(title)

        fig.savefig(f"{path}/05_scores/{curr_score_name}/categorical/{age}/epi_like_acc/all/{age}.png", bbox_inches='tight', dpi=200)
        fig.savefig(f"{path}/05_scores/{curr_score_name}/categorical/{age}/epi_like_acc/all/{age}.pdf", bbox_inches='tight')
        plt.close(fig)

C:\Users\alena\AppData\Local\Temp\ipykernel_38988\2303238152.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_age_scores[f"{age}_linear_pred_cx"] = linreg_cx.predict(df_age_scores)
C:\Users\alena\AppData\Local\Temp\ipykernel_38988\2303238152.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_age_scores[f"{age}_acceleration_cx"] = df_age_scores[age] - df_age_scores[f"{age}_linear_pred_cx"]
C:\Users\alena\AppData\Local\Temp\ipykernel_38988\2303238152.py:20: SettingWithCopyWarning: 
A value is tryi

In [16]:
# Here we take samples with low risk for linreg
ages = ['PhenoAge', 'PhenoAge nmk', 'CognitiveAge']

for curr_score in pheno_associations:
    curr_score_name = curr_score.split(' group')[0]
    df_stat = pd.DataFrame(index=ages)
    score_vals = pheno_associations[curr_score]['groups']
    score_val_base = pheno_associations[curr_score]['base']
    score_colors = pheno_associations[curr_score]['colors']
    
    for age in ages:

        pathlib.Path(f"{path}/05_scores/{curr_score_name}/categorical/{age}/epi_like_acc/low_risk").mkdir(parents=True, exist_ok=True)
        
        df_age_scores = df.dropna(subset=[age, curr_score])

        linreg_cx = smf.ols(formula=f"Q('{age}') ~ Q('Chronological Age ({age})')", data=df_age_scores).fit()
        df_age_scores[f"{age}_linear_pred_cx"] = linreg_cx.predict(df_age_scores)
        df_age_scores[f"{age}_acceleration_cx"] = df_age_scores[age] - df_age_scores[f"{age}_linear_pred_cx"]
        df_age_scores[f"{age}"] = df_age_scores[f'Chronological Age ({age})'] + df_age_scores[f"{age}_acceleration_cx"]

        linreg = smf.ols(formula=f"Q('{age}') ~ Q('Chronological Age ({age})')", data=df_age_scores.loc[df_age_scores[curr_score] == score_val_base, :]).fit()
        df_age_scores[f"{age} linear pred"] = linreg.predict(df_age_scores)
        df_age_scores[f"{age} acceleration"] = df_age_scores[age] - df_age_scores[f"{age} linear pred"]
        vals = {}
        for group in score_vals:
            vals[group] = df_age_scores.loc[df_age_scores[curr_score] == group, f"{age} acceleration"].values
        _, df_stat.at[age, "mannwhitneyu_pval"] = mannwhitneyu(vals[score_vals[0]], vals[score_vals[1]], alternative='two-sided')
        _, df_stat.at[age, "levene_pval"] = levene(vals[score_vals[0]], vals[score_vals[1]])
        regcov = smf.ols(formula=f"Q('{age}') ~ Q('{curr_score}') + Age", data=df_age_scores).fit()
        reg_sum = regcov.summary2().tables[1]
        pvals_cols = reg_sum.index[reg_sum.index.str.contains(curr_score_name)].values
        for pval_col_id, pval_col in enumerate(pvals_cols):
            df_stat.at[age, f"ancova_{pval_col}_pval"] = reg_sum.at[pval_col, 'P>|t|']

        hist_bins = np.linspace(5, 115, 23)
        fig, ax = plt.subplots(figsize=(6, 4))
        histplot = sns.histplot(
            data=df_age_scores,
            bins=hist_bins,
            edgecolor='k',
            linewidth=1,
            x="Age",
            hue=curr_score,
            palette=score_colors,
            ax=ax
        )
        histplot.set(xlim=(0, 120))
        plt.savefig(f"{path}/05_scores/{curr_score_name}/categorical/{age}/epi_like_acc/low_risk/hist_age_corrected.png", bbox_inches='tight', dpi=200)
        plt.savefig(f"{path}/05_scores/{curr_score_name}/categorical/{age}/epi_like_acc/low_risk/hist_age_corrected.pdf", bbox_inches='tight')
        plt.close(fig)

        fig, axs = plt.subplot_mosaic(
            [
                ['11', '12'],
                ['21', '22'],
            ],
            figsize=(8, 6),
            height_ratios=[1, 4],
            width_ratios=[3, 1.5],
            gridspec_kw={
                "bottom": 0.14,
                "top": 0.95,
                "wspace": 0.33,
                "hspace": 0.01,
            },
        )
        
        ds_table = pd.DataFrame(index=[fr"Pearson $\rho$", "Pearson p-value", fr"Spearman $\rho$", "Spearman p-value"], columns=[age])
        rho, pval = pearsonr(df_age_scores.loc[df_age_scores[curr_score] == score_val_base, :][f'Chronological Age ({age})'].values, df_age_scores.loc[df_age_scores[curr_score] == score_val_base, :][age].values)
        sp_rho, sp_pval = spearmanr(df_age_scores.loc[df_age_scores[curr_score] == score_val_base, :][f'Chronological Age ({age})'].values, df_age_scores.loc[df_age_scores[curr_score] == score_val_base, :][age].values)
        ds_table.at[fr"Pearson $\rho$", age] = f"{rho:0.2f}"
        ds_table.at["Pearson p-value", age] = f"{pval:.2e}"
        ds_table.at[fr"Spearman $\rho$", age] = f"{sp_rho:0.2f}"
        ds_table.at["Spearman p-value", age] = f"{sp_pval:.2e}"
        col_defs = [
            ColumnDefinition(
                name="index",
                title=age,
                textprops={"ha": "left"},
                width=4.5,
            ),
            ColumnDefinition(
                name=age,
                title='',
                textprops={"ha": "center"},
                width=2.0,
            ),
        ]
        table = Table(
            ds_table,
            column_definitions=col_defs,
            row_dividers=True,
            footer_divider=False,
            ax=axs['11'],
            textprops={"fontsize": 7},
            row_divider_kw={"linewidth": 1, "linestyle": (0, (1, 1))},
            col_label_divider_kw={"linewidth": 1, "linestyle": "-"},
            column_border_kw={"linewidth": 1, "linestyle": "-"},
        ).autoset_fontcolors(colnames=[age])
        
        axs['12'].axis('off')
        
        xy_min = df_age_scores[[f'Chronological Age ({age})', age]].min().min()
        xy_max = df_age_scores[[f'Chronological Age ({age})', age]].max().max()
        xy_ptp = xy_max - xy_min
        bisect = sns.lineplot(
            x=[xy_min - 0.1 * xy_ptp, xy_max + 0.1 * xy_ptp],
            y=[xy_min - 0.1 * xy_ptp, xy_max + 0.1 * xy_ptp],
            linestyle='--',
            color='black',
            linewidth=1.0,
            ax=axs['21']
        )
        regplot = sns.regplot(
            data=df_age_scores.loc[df_age_scores[curr_score] == score_val_base, :],
            x=f'Chronological Age ({age})',
            y=age,
            color=score_colors[score_val_base],
            scatter=False,
            truncate=False,
            ax=axs['21']
        )
        scatter = sns.scatterplot(
            data=df_age_scores,
            x=f'Chronological Age ({age})',
            y=age,
            hue=curr_score,
            palette=score_colors,
            linewidth=0.5,
            alpha=0.75,
            edgecolor="k",
            s=20,
            hue_order=list(score_colors.keys()),
            legend=True,
            ax=axs['21'],
        )
        axs['21'].set_xlabel(f"Age")
        axs['21'].set_xlim(xy_min - 0.1 * xy_ptp, xy_max + 0.1 * xy_ptp)
        axs['21'].set_ylim(xy_min - 0.1 * xy_ptp, xy_max + 0.1 * xy_ptp)
        scatter.legend_.set_title(curr_score_name)
        
        sns.violinplot(
            data=df_age_scores,
            x=curr_score,
            y=f"{age} acceleration",
            hue=curr_score,
            palette=score_colors,
            density_norm='width',
            order=score_vals,
            saturation=0.75,
            linewidth=1.0,
            ax=axs['22'],
            legend=False,
            cut=0,
        )
        axs['22'].set_ylabel(f"{age} acceleration")
        axs['22'].set_xlabel(curr_score_name)
        mannwhitneyu_pval = df_stat.at[age, "mannwhitneyu_pval"]
        levene_pval = df_stat.at[age, "levene_pval"]
        title = f'Mann-Whitney: {mannwhitneyu_pval:.2e}\nLevene: {levene_pval:.2e}'
        for pval_col in pvals_cols:
            title += f"\nANCOVA: {df_stat.at[age, f'ancova_{pval_col}_pval']:.2e}"
        axs['22'].set_title(title)

        fig.savefig(f"{path}/05_scores/{curr_score_name}/categorical/{age}/epi_like_acc/low_risk/{age}.png", bbox_inches='tight', dpi=200)
        fig.savefig(f"{path}/05_scores/{curr_score_name}/categorical/{age}/epi_like_acc/low_risk/{age}.pdf", bbox_inches='tight')
        plt.close(fig)

C:\Users\alena\AppData\Local\Temp\ipykernel_38988\843357804.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_age_scores[f"{age}_linear_pred_cx"] = linreg_cx.predict(df_age_scores)
C:\Users\alena\AppData\Local\Temp\ipykernel_38988\843357804.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_age_scores[f"{age}_acceleration_cx"] = df_age_scores[age] - df_age_scores[f"{age}_linear_pred_cx"]
C:\Users\alena\AppData\Local\Temp\ipykernel_38988\843357804.py:20: SettingWithCopyWarning: 
A value is trying 

## Ages (continuous score)

In [17]:
# Here we take all samples for linreg
ages = ['PhenoAge', 'PhenoAge nmk', 'CognitiveAge']

for curr_score in pheno_associations:
    curr_score_name = curr_score.split(' group')[0]
    df_stat = pd.DataFrame(index=ages)
    score_vals = pheno_associations[curr_score]['groups']
    score_val_base = pheno_associations[curr_score]['base']
    score_colors = pheno_associations[curr_score]['colors']
    
    for age in ages:

        pathlib.Path(f"{path}/05_scores/{curr_score_name}/continuous/{age}/all").mkdir(parents=True, exist_ok=True)
        
        df_age_scores = df.dropna(subset=[age, curr_score])

        linreg = smf.ols(formula=f"Q('{age}') ~ Q('Chronological Age ({age})')", data=df_age_scores).fit()
        df_age_scores[f"{age} linear pred"] = linreg.predict(df_age_scores)
        df_age_scores[f"{age} acceleration"] = df_age_scores[age] - df_age_scores[f"{age} linear pred"]
        vals = {}
        for group in score_vals:
            vals[group] = df_age_scores.loc[df_age_scores[curr_score] == group, f"{age} acceleration"].values
        _, df_stat.at[age, "mannwhitneyu_pval"] = mannwhitneyu(vals[score_vals[0]], vals[score_vals[1]], alternative='two-sided')
        _, df_stat.at[age, "levene_pval"] = levene(vals[score_vals[0]], vals[score_vals[1]])
        regcov = smf.ols(formula=f"Q('{age}') ~ Q('{curr_score}') + Age", data=df_age_scores).fit()
        reg_sum = regcov.summary2().tables[1]
        pvals_cols = reg_sum.index[reg_sum.index.str.contains(curr_score_name)].values
        for pval_col_id, pval_col in enumerate(pvals_cols):
            df_stat.at[age, f"ancova_{pval_col}_pval"] = reg_sum.at[pval_col, 'P>|t|']

        hist_bins = np.linspace(5, 115, 23)
        fig, ax = plt.subplots(figsize=(6, 4))
        histplot = sns.histplot(
            data=df_age_scores,
            bins=hist_bins,
            edgecolor='k',
            linewidth=1,
            x="Age",
            hue=curr_score,
            palette=score_colors,
            ax=ax
        )
        histplot.set(xlim=(0, 120))
        plt.savefig(f"{path}/05_scores/{curr_score_name}/continuous/{age}/all/hist_age_corrected.png", bbox_inches='tight', dpi=200)
        plt.savefig(f"{path}/05_scores/{curr_score_name}/continuous/{age}/all/hist_age_corrected.pdf", bbox_inches='tight')
        plt.close(fig)

        fig, axs = plt.subplot_mosaic(
            [
                ['11', '12'],
                ['21', '22'],
            ],
            figsize=(8, 6),
            height_ratios=[1, 4],
            width_ratios=[3, 1.5],
            gridspec_kw={
                "bottom": 0.14,
                "top": 0.95,
                "wspace": 0.33,
                "hspace": 0.01,
            },
        )
        
        ds_table = pd.DataFrame(index=[fr"Pearson $\rho$", "Pearson p-value", fr"Spearman $\rho$", "Spearman p-value"], columns=[age])
        rho, pval = pearsonr(df_age_scores[curr_score_name].values, df_age_scores[f"{age} acceleration"].values)
        sp_rho, sp_pval = spearmanr(df_age_scores[curr_score_name].values, df_age_scores[f"{age} acceleration"].values)
        ds_table.at[fr"Pearson $\rho$", age] = f"{rho:0.2f}"
        ds_table.at["Pearson p-value", age] = f"{pval:.2e}"
        ds_table.at[fr"Spearman $\rho$", age] = f"{sp_rho:0.2f}"
        ds_table.at["Spearman p-value", age] = f"{sp_pval:.2e}"
        col_defs = [
            ColumnDefinition(
                name="index",
                title=age,
                textprops={"ha": "left"},
                width=4.5,
            ),
            ColumnDefinition(
                name=age,
                title='',
                textprops={"ha": "center"},
                width=2.0,
            ),
        ]
        table = Table(
            ds_table,
            column_definitions=col_defs,
            row_dividers=True,
            footer_divider=False,
            ax=axs['11'],
            textprops={"fontsize": 7},
            row_divider_kw={"linewidth": 1, "linestyle": (0, (1, 1))},
            col_label_divider_kw={"linewidth": 1, "linestyle": "-"},
            column_border_kw={"linewidth": 1, "linestyle": "-"},
        ).autoset_fontcolors(colnames=[age])
        
        axs['12'].axis('off')
        
        x_min = df_age_scores[f"{age} acceleration"].min()
        x_max = df_age_scores[f"{age} acceleration"].max()
        x_ptp = x_max - x_min
        y_min = df_age_scores[curr_score_name].min()
        y_max = df_age_scores[curr_score_name].max()
        y_ptp = y_max - y_min
        regplot = sns.regplot(
            data=df_age_scores,
            x=f'{age} acceleration',
            y=curr_score_name,
            color='dimgray',
            scatter=False,
            ax=axs['21']
        )
        scatter = sns.scatterplot(
            data=df_age_scores,
            x=f"{age} acceleration",
            y=curr_score_name,
            hue=curr_score,
            palette=score_colors,
            linewidth=0.5,
            alpha=0.75,
            edgecolor="k",
            s=20,
            hue_order=list(score_colors.keys()),
            legend=True,
            ax=axs['21'],
        )
        axs['21'].set_xlim(x_min - 0.1 * x_ptp, x_max + 0.1 * x_ptp)
        axs['21'].set_ylim(y_min - 0.1 * y_ptp, y_max + 0.1 * y_ptp)
        scatter.legend_.set_title(curr_score_name)
        
        sns.violinplot(
            data=df_age_scores,
            x=curr_score,
            y=f"{age} acceleration",
            hue=curr_score,
            palette=score_colors,
            density_norm='width',
            order=score_vals,
            saturation=0.75,
            linewidth=1.0,
            ax=axs['22'],
            legend=False,
            cut=0,
        )
        axs['22'].set_ylabel(f"{age} acceleration")
        axs['22'].set_xlabel(curr_score_name)
        mannwhitneyu_pval = df_stat.at[age, "mannwhitneyu_pval"]
        levene_pval = df_stat.at[age, "levene_pval"]
        title = f'Mann-Whitney: {mannwhitneyu_pval:.2e}\nLevene: {levene_pval:.2e}'
        for pval_col in pvals_cols:
            title += f"\nANCOVA: {df_stat.at[age, f'ancova_{pval_col}_pval']:.2e}"
        axs['22'].set_title(title)

        fig.savefig(f"{path}/05_scores/{curr_score_name}/continuous/{age}/all/{age}.png", bbox_inches='tight', dpi=200)
        fig.savefig(f"{path}/05_scores/{curr_score_name}/continuous/{age}/all/{age}.pdf", bbox_inches='tight')
        plt.close(fig)

C:\Users\alena\AppData\Local\Temp\ipykernel_38988\422026921.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_age_scores[f"{age} linear pred"] = linreg.predict(df_age_scores)
C:\Users\alena\AppData\Local\Temp\ipykernel_38988\422026921.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_age_scores[f"{age} acceleration"] = df_age_scores[age] - df_age_scores[f"{age} linear pred"]
C:\Users\alena\AppData\Local\Temp\ipykernel_38988\422026921.py:18: SettingWithCopyWarning: 
A value is trying to be set on

In [18]:
# Here we take samples with low risk for linreg
ages = ['PhenoAge', 'PhenoAge nmk', 'CognitiveAge']

for curr_score in pheno_associations:
    curr_score_name = curr_score.split(' group')[0]
    df_stat = pd.DataFrame(index=ages)
    score_vals = pheno_associations[curr_score]['groups']
    score_val_base = pheno_associations[curr_score]['base']
    score_colors = pheno_associations[curr_score]['colors']
    
    for age in ages:

        pathlib.Path(f"{path}/05_scores/{curr_score_name}/continuous/{age}/low_risk").mkdir(parents=True, exist_ok=True)
        
        df_age_scores = df.dropna(subset=[age, curr_score])

        linreg = smf.ols(formula=f"Q('{age}') ~ Q('Chronological Age ({age})')", data=df_age_scores.loc[df_age_scores[curr_score] == score_val_base, :]).fit()
        df_age_scores[f"{age} linear pred"] = linreg.predict(df_age_scores)
        df_age_scores[f"{age} acceleration"] = df_age_scores[age] - df_age_scores[f"{age} linear pred"]
        vals = {}
        for group in score_vals:
            vals[group] = df_age_scores.loc[df_age_scores[curr_score] == group, f"{age} acceleration"].values
        _, df_stat.at[age, "mannwhitneyu_pval"] = mannwhitneyu(vals[score_vals[0]], vals[score_vals[1]], alternative='two-sided')
        _, df_stat.at[age, "levene_pval"] = levene(vals[score_vals[0]], vals[score_vals[1]])
        regcov = smf.ols(formula=f"Q('{age}') ~ Q('{curr_score}') + Age", data=df_age_scores).fit()
        reg_sum = regcov.summary2().tables[1]
        pvals_cols = reg_sum.index[reg_sum.index.str.contains(curr_score_name)].values
        for pval_col_id, pval_col in enumerate(pvals_cols):
            df_stat.at[age, f"ancova_{pval_col}_pval"] = reg_sum.at[pval_col, 'P>|t|']

        hist_bins = np.linspace(5, 115, 23)
        fig, ax = plt.subplots(figsize=(6, 4))
        histplot = sns.histplot(
            data=df_age_scores,
            bins=hist_bins,
            edgecolor='k',
            linewidth=1,
            x="Age",
            hue=curr_score,
            palette=score_colors,
            ax=ax
        )
        histplot.set(xlim=(0, 120))
        plt.savefig(f"{path}/05_scores/{curr_score_name}/continuous/{age}/low_risk/hist_age_corrected.png", bbox_inches='tight', dpi=200)
        plt.savefig(f"{path}/05_scores/{curr_score_name}/continuous/{age}/low_risk/hist_age_corrected.pdf", bbox_inches='tight')
        plt.close(fig)

        fig, axs = plt.subplot_mosaic(
            [
                ['11', '12'],
                ['21', '22'],
            ],
            figsize=(8, 6),
            height_ratios=[1, 4],
            width_ratios=[3, 1.5],
            gridspec_kw={
                "bottom": 0.14,
                "top": 0.95,
                "wspace": 0.33,
                "hspace": 0.01,
            },
        )
        
        ds_table = pd.DataFrame(index=[fr"Pearson $\rho$", "Pearson p-value", fr"Spearman $\rho$", "Spearman p-value"], columns=[age])
        rho, pval = pearsonr(df_age_scores.loc[df_age_scores[curr_score] == score_val_base, :][curr_score_name].values, df_age_scores.loc[df_age_scores[curr_score] == score_val_base, :][f"{age} acceleration"].values)
        sp_rho, sp_pval = spearmanr(df_age_scores.loc[df_age_scores[curr_score] == score_val_base, :][curr_score_name].values, df_age_scores.loc[df_age_scores[curr_score] == score_val_base, :][f"{age} acceleration"].values)
        ds_table.at[fr"Pearson $\rho$", age] = f"{rho:0.2f}"
        ds_table.at["Pearson p-value", age] = f"{pval:.2e}"
        ds_table.at[fr"Spearman $\rho$", age] = f"{sp_rho:0.2f}"
        ds_table.at["Spearman p-value", age] = f"{sp_pval:.2e}"
        col_defs = [
            ColumnDefinition(
                name="index",
                title=age,
                textprops={"ha": "left"},
                width=4.5,
            ),
            ColumnDefinition(
                name=age,
                title='',
                textprops={"ha": "center"},
                width=2.0,
            ),
        ]
        table = Table(
            ds_table,
            column_definitions=col_defs,
            row_dividers=True,
            footer_divider=False,
            ax=axs['11'],
            textprops={"fontsize": 7},
            row_divider_kw={"linewidth": 1, "linestyle": (0, (1, 1))},
            col_label_divider_kw={"linewidth": 1, "linestyle": "-"},
            column_border_kw={"linewidth": 1, "linestyle": "-"},
        ).autoset_fontcolors(colnames=[age])
        
        axs['12'].axis('off')
        
        x_min = df_age_scores[f"{age} acceleration"].min()
        x_max = df_age_scores[f"{age} acceleration"].max()
        x_ptp = x_max - x_min
        y_min = df_age_scores[curr_score_name].min()
        y_max = df_age_scores[curr_score_name].max()
        y_ptp = y_max - y_min
        regplot = sns.regplot(
            data=df_age_scores.loc[df_age_scores[curr_score] == score_val_base, :],
            x=f'{age} acceleration',
            y=curr_score_name,
            color=score_colors[score_val_base],
            scatter=False,
            ax=axs['21']
        )
        scatter = sns.scatterplot(
            data=df_age_scores,
            x=f"{age} acceleration",
            y=curr_score_name,
            hue=curr_score,
            palette=score_colors,
            linewidth=0.5,
            alpha=0.75,
            edgecolor="k",
            s=20,
            hue_order=list(score_colors.keys()),
            legend=True,
            ax=axs['21'],
        )
        axs['21'].set_xlim(x_min - 0.1 * x_ptp, x_max + 0.1 * x_ptp)
        axs['21'].set_ylim(y_min - 0.1 * y_ptp, y_max + 0.1 * y_ptp)
        scatter.legend_.set_title(curr_score_name)
        
        sns.violinplot(
            data=df_age_scores,
            x=curr_score,
            y=f"{age} acceleration",
            hue=curr_score,
            palette=score_colors,
            density_norm='width',
            order=score_vals,
            saturation=0.75,
            linewidth=1.0,
            ax=axs['22'],
            legend=False,
            cut=0,
        )
        axs['22'].set_ylabel(f"{age} acceleration")
        axs['22'].set_xlabel(curr_score_name)
        mannwhitneyu_pval = df_stat.at[age, "mannwhitneyu_pval"]
        levene_pval = df_stat.at[age, "levene_pval"]
        title = f'Mann-Whitney: {mannwhitneyu_pval:.2e}\nLevene: {levene_pval:.2e}'
        for pval_col in pvals_cols:
            title += f"\nANCOVA: {df_stat.at[age, f'ancova_{pval_col}_pval']:.2e}"
        axs['22'].set_title(title)

        fig.savefig(f"{path}/05_scores/{curr_score_name}/continuous/{age}/low_risk/{age}.png", bbox_inches='tight', dpi=200)
        fig.savefig(f"{path}/05_scores/{curr_score_name}/continuous/{age}/low_risk/{age}.pdf", bbox_inches='tight')
        plt.close(fig)

C:\Users\alena\AppData\Local\Temp\ipykernel_38988\4061469146.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_age_scores[f"{age} linear pred"] = linreg.predict(df_age_scores)
C:\Users\alena\AppData\Local\Temp\ipykernel_38988\4061469146.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_age_scores[f"{age} acceleration"] = df_age_scores[age] - df_age_scores[f"{age} linear pred"]
C:\Users\alena\AppData\Local\Temp\ipykernel_38988\4061469146.py:18: SettingWithCopyWarning: 
A value is trying to be set